In [ ]:
from typing import Dict, List, Any, Optional
from dataclasses import dataclass
import openai
import json
from enum import Enum
from datetime import datetime
import logging
from dotenv import load_dotenv
import os
from openai import OpenAI
import ast
from pprint import pprint
from openai import AsyncOpenAI
import asyncio

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

@dataclass
class QueryContext:
    original_query: str
    timestamp: str
    query_id: str
    
@dataclass
class SubProblem:
    category: str
    description: str
    dependencies: List[str]
    priority: int
    context_reference: Dict[str, Any]
    query_segment: str

@dataclass
class Solution:
    sub_problem: SubProblem
    proposed_solution: Dict
    confidence: float
    reasoning: str
    validation_notes: List[str]

class Agent:
    def __init__(self, api_key: str):
        self.client = OpenAI(api_key=api_key)
        self.async_client = AsyncOpenAI(api_key=api_key)
        
    def call_llm(self, messages: List[Dict]) -> str:
        try:
            print(json.dumps(messages, indent=2))
            response = self.client.chat.completions.create(
                model="gpt-4o",
                messages=messages,
                temperature=0.7
            )
            print((response.choices[0].message.content))
            return response.choices[0].message.content
        except Exception as e:
            logger.error(f"Error in LLM call: {e}")
            raise

    async def call_llm_async(self, messages: List[Dict]) -> str:
        try:
            response = await self.async_client.chat.completions.create(
                model="gpt-4o",
                messages=messages,
                temperature=0.7
            )
            return response.choices[0].message.content
        except Exception as e:
            logger.error(f"Error in async LLM call: {e}")
            raise

class IntentAnalysisAgent(Agent):
    def __init__(self, api_key: str):
        super().__init__(api_key)
        self.system_prompt = """You are an expert recruitment query analyzer specializing in breaking down complex queries into logical sub-problems.

        Your primary responsibilities:
        1. Deeply understand the main intent of the recruitment query
        2. Break down the query into distinct, manageable sub-problems
        3. Identify and maintain relationships between sub-problems
        4. Preserve context and references to the original query
        5. Assign priority levels (1-5, 1 being highest)
        
        Think through these steps:
        1. What is the core requirement?
        2. What are the mandatory vs optional criteria?
        3. Are there implicit requirements that need to be made explicit?
        4. How do different parts of the query relate to each other?
        5. What potential ambiguities need to be resolved?

        For each sub-problem, consider:
        - Which part of the original query does it address?
        - What other sub-problems might it depend on?
        - What specific filter categories does it relate to?
        - What potential challenges might arise in solving it?


        Output JSON structure:
        {
            "main_intent": "Clear statement of primary goal",
            "thinking" : "",
            "sub_problems": [
                {
                    "category": "job_title/location/education/etc",
                    "description": "Detailed description",
                    "dependencies": ["other_category_ids"],
                    "priority": "1-5",
                    "context_reference": {"query_segment": "relevant part of original query"},
                    "query_segment": "exact text from original query"
                }
            ],
            "analysis_notes": ["Detailed thoughts and reasoning"],
            "potential_challenges": ["Identified challenges"],
            "confidence_score": 0.0
        }"""

    def analyze(self, query_context: QueryContext) -> Dict:
        prompt = f"""Analyze this recruitment query with extreme attention to detail: {query_context.original_query}

        Consider these specific aspects:
        1. What are the explicit requirements?
        2. What are the implicit requirements?
        3. Are there any ambiguous terms that need clarification?
        4. How do different requirements relate to each other?
        
        Show your reasoning process."""

        messages = [
            {"role": "system", "content": self.system_prompt},
            {"role": "user", "content": prompt}
        ]
        
        response = self.call_llm(messages)
        response = eval(response[response.find('{'):response.rfind('}') + 1])
        return (response)

class SubProblemAnalysisAgent(Agent):
    def __init__(self, api_key: str):
        super().__init__(api_key)
        self.system_prompt = """You are an expert in solving specific recruitment sub-problems.
        IMPORTANT: Focus ONLY on the specific sub-problem assigned to you. While you may reference
        the original query for context, your solution should be specifically targeted to your assigned sub-problem.

        For this specific sub-problem, follow this detailed analysis process:
        1. Initial Assessment
           - Understand the specific requirement of THIS sub-problem
           - Identify relevant filter categories for THIS component
           - Review dependencies and constraints specific to THIS aspect

        2. Solution Generation
           - Generate multiple potential approaches for THIS specific requirement
           - Consider edge cases and limitations
           - Ensure solution stays focused on the assigned sub-problem
           - Solution should be so that it is only to generate a specific filter value
        
        Note: While you are generating proposed solutions, you need to think mostly in terms of using which filter and what possible values can be used for that filter.

        3. Solution Evaluation
           - Score each solution (1-10)
           - List pros and cons specific to this sub-problem
           - Consider implementation feasibility

        4. Final Selection
           - Choose optimal solution for this specific component
           - Document reasoning
           - Provide confidence score

        Remember: Stay focused on your assigned sub-problem. Don't try to solve the entire query.

        Output structure:
        {
            "sub_problem_analysis": {
                "original_context": "",
                "interpreted_requirement": "",
                "thought_process": "",
                "proposed_solutions": [
                    {
                        "thinking" : "",
                        "approach": "",
                        "score": 0,
                        "pros": [],
                        "cons": [],
                        "implementation_notes": ""
                    }
                ],
                "selected_solution": {
                    "thinking_for_selection" : "",
                    "filter_values": {},
                    "confidence": 0.0,
                    "reasoning": "",
                    "validation_notes": []
                }
            }
        }"""

    async def solve_async(self, sub_problem: SubProblem, query_context: QueryContext) -> Solution:
        prompt = f"""Focus on solving this specific sub-problem:

        Sub-problem Category: {sub_problem.category}
        Sub-problem Description: {sub_problem.description}
        Specific Query Segment to Address: {sub_problem.query_segment}

        For reference only, original query context: {query_context.original_query}

        Remember: Focus only on solving this specific sub-problem component.
        Do not attempt to solve the entire query.

        Show your detailed reasoning process for this specific component."""

        messages = [
            {"role": "system", "content": self.system_prompt},
            {"role": "user", "content": prompt}
        ]
        
        response = await self.call_llm_async(messages)
        response = (response[response.find('{'):response.rfind('}') + 1])
        return json.loads(response)

class IntegrationAgent(Agent):
    def __init__(self, api_key: str):
        super().__init__(api_key)
        self.system_prompt = """You are an expert in integrating and validating recruitment filter solutions and converting them into specific JSON filter formats.

        Your critical responsibilities:
        1. Analyze all solutions thoroughly
        2. Provide detailed reasoning for each decision
        3. Convert the solutions into the exact filter format
        4. Only include relevant filters with valid values
        5. Ensure all values are logically consistent

        The final output must strictly follow this filter structure:
        {
            "job_title/business_function": {
                "current": [{"title_name": "", "min_staff": 0, "max_staff": 50000000}],
                "past": [{"title_name": "", "min_staff": 0, "max_staff": 50000000}],
                "both": [{"title_name": "", "min_staff": 0, "max_staff": 50000000}],
                "event": ""  # Can only be "CURRENT", "PAST", "CURRENT OR PAST", "CURRENT AND PAST"
            },
            "management_level": {
                "current": [],
                "past": [],
                "both": [],
                "event": ""
            },
            "location": {
                "current": [],
                "past": [],
                "both": [],
                "event": ""
            },
            "company": {
                "current_prompt": "",
                "past_prompt": "",
                "event": ""
            },
            "education": [{"degree": "", "major": ""}],
            "school": [],
            "current_ownership": [],
            "name": [],
            "ethnicity": [],
            "age": [],
            "total_working_years": {"min": null, "max": null},
            "role_tenure": {"min": null, "max": null},
            "company_tenure": {"min": null, "max": null},
            "skills": []
        }

        Before providing the final JSON:
        1. Analyze each requirement thoroughly
        2. Explain your reasoning for each filter value
        3. Discuss any assumptions or implicit requirements
        4. Validate logical consistency
        5. Only include filters that have valid values
        6. Explain any complex decisions or trade-offs

        Your response should follow this structure:
        1. DETAILED ANALYSIS
        2. REASONING FOR EACH FILTER
        3. ASSUMPTIONS AND IMPLICATIONS
        4. VALIDATION CHECKS
        5. FINAL FILTER JSON
        """

    def integrate(self, solutions: List[Solution], query_context: QueryContext) -> Dict:
        print("#"*47)
        print(solutions)
        print("#"*47)
        prompt = f"""Integrate these solutions while maintaining alignment with the original query:

        Original Query: {query_context.original_query}

        Please provide:
        1. Detailed analysis of how each solution component fits into the final filter structure
        2. Explicit reasoning for each filter value being set
        3. Discussion of any assumptions or implicit requirements
        4. Explanation of any trade-offs or decisions made
        5. Final filter JSON in the exact specified format

        Number of Solutions to Integrate: {len(solutions)}

        Solutions: {solutions}

        For each solution component, think through:
        - How does this map to our filter structure?
        - What are the explicit vs implicit requirements?
        - Are there any temporal aspects to consider (current vs past)?
        - What are the logical dependencies between different filters?
        - Are there any potential conflicts to resolve?

        Show your complete reasoning process before providing the final JSON structure.
        """

        messages = [
            {"role": "system", "content": self.system_prompt},
            {"role": "user", "content": prompt}
        ]
        
        response = self.call_llm(messages)
        # print("Integrated Solutions...")
        # print(response)
        json_str = (response[response.find('{'):response.rfind('}') + 1])
        # print("JSON String...")
        # print(json_str)

        
        # Split the response into reasoning and JSON parts
        parts = response.split("FINAL FILTER JSON:")
        reasoning = parts[0].strip()

        
        # Parse the JSON string

        filter_structure = json.loads(json_str)
        
        return {
            "reasoning_and_analysis": reasoning,
            "filter_structure": filter_structure
        }

# Modify RecruitmentQueryProcessor
class RecruitmentQueryProcessor:
    def __init__(self, api_key: str):
        self.intent_agent = IntentAnalysisAgent(api_key)
        self.subproblem_agent = SubProblemAnalysisAgent(api_key)
        self.integration_agent = IntegrationAgent(api_key)

    async def process_sub_problems(self, sub_problems: List[Dict], query_context: QueryContext) -> List[Solution]:
        # Create tasks for all sub-problems
        tasks = []
        for sub_problem_dict in sub_problems:
            sub_problem = SubProblem(**sub_problem_dict)
            task = self.subproblem_agent.solve_async(sub_problem, query_context)
            tasks.append(task)
        
        # Execute all tasks concurrently
        solutions = await asyncio.gather(*tasks)
        return solutions

    async def process_query_async(self, query: str) -> Dict:
        query_context = QueryContext(
            original_query=query,
            timestamp=datetime.now().isoformat(),
            query_id=f"query_{hash(query)}"
        )
        
        logger.info(f"Processing query: {query_context.query_id}")

        try:
            # Step 1: Analyze intent and decompose
            print('-'*47)
            print("Analyzing Intent and Decomposing the Problem...")
            analysis = self.intent_agent.analyze(query_context)
            logger.info(f"Query decomposed into {len(analysis['sub_problems'])} sub-problems")

            # Step 2: Process all sub-problems concurrently
            print('-'*47)
            print("Processing the Sub-Problems...")
            solutions = await self.process_sub_problems(analysis['sub_problems'], query_context)
            logger.info(f"Generated solutions for all sub-problems")

            # Step 3: Integrate solutions
            print('-'*47)
            print("Integrating Solutions of Sub-Problems...")
            final_result = self.integration_agent.integrate(solutions, query_context)
            logger.info("Solutions integrated successfully")

            print('-'*47)
            print("Printing Final Result...")
            print(json.dumps(final_result, indent=2))


            return {
                "query_context": vars(query_context),
                "analysis": analysis,
                "solutions": solutions,
                "final_result": final_result
            }

        except Exception as e:
            logger.error(f"Error processing query: {e}")
            raise

# Modify main function
async def main():
    _ = load_dotenv()
    api_key = os.getenv("OPENAI_API_KEY")
    processor = RecruitmentQueryProcessor(api_key)
    
    query = "Find me a CEO from Microsoft or similar companies with an MBA degree and at least 15 years of experience"
    queries = [
    "Find sales executives.",
    "Find VP-level leaders from companies in Western Europe with over 30 years of experience and expertise in corporate finance.",
    "Generate a precise list of individuals currently serving as President, CEO, or COO in companies similar to Jawbone within the wearable technology sector. These companies focus on products such as fitness trackers, smartwatches, health monitoring devices, or other wearable electronics but do not necessarily have to manufacture them. The individuals should be based in the United States or Europe and hold an MBA degree. Exclude individuals from companies not focused on wearable technology. Precision is important",
    "Find me a Chief Marketing Officer with a strong track record in scaling AI-driven SaaS platforms. Preferably based in the U.S.",
    "Executive Vice President and General Manager at companies specializing in architectural lighting solutions, commercial illumination systems, and smart building technologies. Expertise in LED lighting, energy-efficient systems, and IoT-enabled lighting controls. Proficient in strategic leadership, operational management, and business development within the construction and real estate sectors. Skilled in driving revenue growth, optimizing supply chain logistics, and enhancing customer experience. Experience in managing large-scale projects, overseeing product innovation, and leading cross-functional teams. Familiar with industry standards, sustainability practices, and regulatory compliance.",
    ]
    queries = ["Find VP-level leaders from AI companies in Western Europe with over 30 years of experience and expertise in corporate finance who have worked previously as Salesman."]
    query_results = []
    for query in queries:
        print('-'*47)
        print("QUERY: ", query)
        
        try:
            result = await processor.process_query_async(query)
            query_results.append({"query": query, "result" : result})
        except:
            pass
    
    return query_results


results = await (main())

with open("results_1.json", "w") as f:
    json.dump(results, f, indent=2)

INFO:__main__:Processing query: query_6766064356711010706


-----------------------------------------------
QUERY:  Find VP-level leaders from AI companies in Western Europe with over 30 years of experience and expertise in corporate finance who have worked previously as Salesman.
-----------------------------------------------
Analyzing Intent and Decomposing the Problem...
[
  {
    "role": "system",
    "content": "You are an expert recruitment query analyzer specializing in breaking down complex queries into logical sub-problems.\n\n        Your primary responsibilities:\n        1. Deeply understand the main intent of the recruitment query\n        2. Break down the query into distinct, manageable sub-problems\n        3. Identify and maintain relationships between sub-problems\n        4. Preserve context and references to the original query\n        5. Assign priority levels (1-5, 1 being highest)\n        \n        Think through these steps:\n        1. What is the core requirement?\n        2. What are the mandatory vs optional criteri

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:__main__:Query decomposed into 6 sub-problems


```json
{
    "main_intent": "Identify VP-level leaders in AI companies with specific experience and expertise in Western Europe.",
    "thinking": "The query seeks a specific profile of individuals who possess leadership experience at a VP level, a background in AI, extensive experience in corporate finance, and past roles in salesmanship, all within a geographical constraint of Western Europe.",
    "sub_problems": [
        {
            "category": "job_title",
            "description": "Identify candidates holding or having held VP-level roles.",
            "dependencies": ["company_industry"],
            "priority": 1,
            "context_reference": {"query_segment": "Find VP-level leaders"},
            "query_segment": "VP-level leaders"
        },
        {
            "category": "company_industry",
            "description": "Focus on candidates from AI companies.",
            "dependencies": ["job_title"],
            "priority": 2,
            "context_reference": {"

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 500 Internal Server Error"
INFO:openai._base_client:Retrying request to /chat/completions in 0.462600 seconds
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:__main__:Generated solutions for all sub-problems


-----------------------------------------------
Integrating Solutions of Sub-Problems...
###############################################
[{'sub_problem_analysis': {'original_context': 'Find VP-level leaders from AI companies in Western Europe with over 30 years of experience and expertise in corporate finance who have worked previously as Salesman.', 'interpreted_requirement': 'Identify candidates holding or having held VP-level roles.', 'thought_process': 'The task is to filter candidates based on their job titles to identify those who are or have been in VP-level positions. This requires understanding the hierarchy of job titles and determining which titles correspond to VP-level positions. This filter will help in narrowing down the pool of candidates to those who are likely to meet the leadership criteria set by the requester.', 'proposed_solutions': [{'thinking': 'Identify specific job titles that are commonly associated with VP-level roles.', 'approach': "Filter candidates by job

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:__main__:Solutions integrated successfully


### 1. DETAILED ANALYSIS

Let's analyze each component of the solutions and how they fit into the filter structure:

**a. Job Title/Business Function:**
- The primary focus is on identifying VP-level roles. The solutions suggest specific titles such as "Vice President," "VP," "Senior Vice President," and "Executive Vice President."
- Additionally, the requirement is to identify those who have previously worked as a "Salesman" or similar roles like "Sales Representative," "Sales Associate," and "Sales Executive."
- The temporal aspect is important here, as we need to distinguish between current roles (VP-level) and past roles (Salesman).

**b. Company:**
- The focus is on candidates associated with AI companies. A list of well-known AI companies, such as "OpenAI," "DeepMind," and others, is provided.
- The requirement is to consider candidates who are currently or have previously been employed by these companies.

**c. Location:**
- The candidates must be located in Western Europe. A li

In [1]:
from typing import Dict, List, Any, Optional
from dataclasses import dataclass
import openai
import json
from enum import Enum
from datetime import datetime
import logging
from dotenv import load_dotenv
import os
from openai import OpenAI
import ast
from pprint import pprint
from openai import AsyncOpenAI
import asyncio

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

@dataclass
class QueryContext:
    original_query: str
    timestamp: str
    query_id: str
    feedback: Optional[str] = None  # Added feedback field
    
@dataclass
class SubProblem:
    category: str
    description: str
    dependencies: List[str]
    priority: int
    context_reference: Dict[str, Any]
    query_segment: str
    relevant_feedback: Optional[str] = None  # Added relevant feedback field

@dataclass
class Solution:
    sub_problem: SubProblem
    proposed_solution: Dict
    confidence: float
    reasoning: str
    validation_notes: List[str]

class Agent:
    def __init__(self, api_key: str):
        self.client = OpenAI(api_key=api_key)
        self.async_client = AsyncOpenAI(api_key=api_key)
        
    def call_llm(self, messages: List[Dict]) -> str:
        try:
            print(json.dumps(messages, indent=2))
            response = self.client.chat.completions.create(
                model="gpt-4o",
                messages=messages,
                temperature=0.7
            )
            print((response.choices[0].message.content))
            return response.choices[0].message.content
        except Exception as e:
            logger.error(f"Error in LLM call: {e}")
            raise

    async def call_llm_async(self, messages: List[Dict]) -> str:
        try:
            response = await self.async_client.chat.completions.create(
                model="gpt-4o",
                messages=messages,
                temperature=0.7
            )
            return response.choices[0].message.content
        except Exception as e:
            logger.error(f"Error in async LLM call: {e}")
            raise

class FeedbackAwareIntentAnalysisAgent(Agent):
    def __init__(self, api_key: str):
        super().__init__(api_key)
        self.system_prompt = """You are an expert recruitment query analyzer specializing in breaking down complex queries into logical sub-problems while incorporating feedback.

        Your primary responsibilities:
        1. Deeply understand the main intent of the recruitment query and the feedback
        2. Analyze and incorporate any provided feedback
        3. Break down the query into distinct, manageable sub-problems according to the provided feedback
        4. Identify and maintain relationships between sub-problems based on the feedback
        5. Preserve context and references to the original query and feedback
        6. Assign priority levels (1-5, 1 being highest)
        7. Connect relevant parts of the feedback to specific sub-problems
        8. While breaking down the query into sub-problems, your main focus should be to focus on the provided feedback.
        
        Think through these steps:
        1. What is the core requirement?
        2. What are the mandatory vs optional criteria?
        3. Are there implicit requirements that need to be made explicit?
        4. How do different parts of the query relate to each other?
        5. What potential ambiguities need to be resolved?
        6. How does the feedback modify, clarify, or redirect the original query?
        7. Which specific feedback points apply to which sub-problems?

        For each sub-problem, consider:
        - Which part of the original query does it address?
        - What other sub-problems might it depend on?
        - What specific filter categories does it relate to?
        - What potential challenges might arise in solving it?
        - What specific feedback points are relevant to this sub-problem?
        - How should the feedback influence this sub-problem's solution?

        Output JSON structure:
        {
            "main_intent": "Clear statement of primary goal",
            "feedback_analysis": "Analysis of how feedback modifies the query",
            "thinking" : "",
            "sub_problems": [
                {
                    "category": "job_title/location/education/etc",
                    "description": "Detailed description",
                    "dependencies": ["other_category_ids"],
                    "priority": "1-5",
                    "context_reference": {"query_segment": "relevant part of original query"},
                    "query_segment": "exact text from original query",
                    "relevant_feedback": "feedback specifically relevant to this sub-problem"
                }
            ],
            "analysis_notes": ["Detailed thoughts and reasoning"],
            "potential_challenges": ["Identified challenges"],
            "confidence_score": 0.0
        }"""

    def analyze(self, query_context: QueryContext) -> Dict:
        feedback_section = ""
        if query_context.feedback:
            feedback_section = f"""
            The following feedback has been provided about previous attempts to solve this query:
            {query_context.feedback}
            
            When analyzing the query, carefully incorporate this feedback to improve the decomposition.
            For each sub-problem, identify specific parts of the feedback that apply to it.
            """
        
        prompt = f"""Analyze this recruitment query with extreme attention to detail: {query_context.original_query}

        {feedback_section}

        Consider these specific aspects:
        1. What are the explicit requirements?
        2. What are the implicit requirements?
        3. Are there any ambiguous terms that need clarification?
        4. How do different requirements relate to each other?
        5. How does the feedback modify our understanding of the requirements?
        
        Show your reasoning process."""

        messages = [
            {"role": "system", "content": self.system_prompt},
            {"role": "user", "content": prompt}
        ]
        
        response = self.call_llm(messages)
        response = eval(response[response.find('{'):response.rfind('}') + 1])
        return response

class FeedbackAwareSubProblemAnalysisAgent(Agent):
    def __init__(self, api_key: str):
        super().__init__(api_key)
        self.system_prompt = """You are an expert in solving specific recruitment sub-problems, with the ability to incorporate targeted feedback.
        IMPORTANT: Focus ONLY on the specific sub-problem assigned to you. While you may reference
        the original query for context, your solution should be specifically targeted to your assigned sub-problem.

        For this specific sub-problem, follow this detailed analysis process:
        1. Initial Assessment
           - Understand the specific requirement of THIS sub-problem
           - Review any specific feedback provided for this sub-problem
           - Identify relevant filter categories for THIS component
           - Review dependencies and constraints specific to THIS aspect

        2. Feedback Integration
           - Carefully analyze how the provided feedback applies to this specific sub-problem
           - Adjust your approach based on the feedback
           - Address any issues or improvements mentioned in the feedback

        3. Solution Generation
           - Generate multiple potential approaches for THIS specific requirement
           - Consider edge cases and limitations
           - Ensure solution stays focused on the assigned sub-problem
           - Solution should be so that it is only to generate a specific filter value
           - Apply the lessons learned from feedback
        
        Note: While you are generating proposed solutions, you need to think mostly in terms of using which filter and what possible values can be used for that filter.

        4. Solution Evaluation
           - Score each solution (1-10)
           - List pros and cons specific to this sub-problem
           - Consider implementation feasibility
           - Evaluate how well the solution addresses the feedback

        5. Final Selection
           - Choose optimal solution for this specific component
           - Document reasoning
           - Provide confidence score
           - Explain how feedback influenced your final choice

        Remember: Stay focused on your assigned sub-problem. Don't try to solve the entire query.

        Output structure:
        {
            "sub_problem_analysis": {
                "original_context": "",
                "feedback_integration": "",
                "interpreted_requirement": "",
                "thought_process": "",
                "proposed_solutions": [
                    {
                        "thinking" : "",
                        "approach": "",
                        "score": 0,
                        "pros": [],
                        "cons": [],
                        "implementation_notes": "",
                        "feedback_addressed": ""
                    }
                ],
                "selected_solution": {
                    "thinking_for_selection" : "",
                    "filter_values": {},
                    "confidence": 0.0,
                    "reasoning": "",
                    "validation_notes": [],
                    "feedback_influence": ""
                }
            }
        }"""

    async def solve_async(self, sub_problem: SubProblem, query_context: QueryContext) -> Solution:
        feedback_section = ""
        if sub_problem.relevant_feedback:
            feedback_section = f"""
            The following feedback is specifically relevant to this sub-problem:
            {sub_problem.relevant_feedback}
            
            Make sure your solution directly addresses this feedback.
            """
        elif query_context.feedback:
            feedback_section = f"""
            General feedback on the query (consider only what's relevant to this sub-problem):
            {query_context.feedback}
            """

        prompt = f"""Focus on solving this specific sub-problem:

        Sub-problem Category: {sub_problem.category}
        Sub-problem Description: {sub_problem.description}
        Specific Query Segment to Address: {sub_problem.query_segment}

        {feedback_section}

        For reference only, original query context: {query_context.original_query}

        Remember: Focus only on solving this specific sub-problem component.
        Do not attempt to solve the entire query.

        Show your detailed reasoning process for this specific component,
        including how you've incorporated the relevant feedback."""

        messages = [
            {"role": "system", "content": self.system_prompt},
            {"role": "user", "content": prompt}
        ]
        
        response = await self.call_llm_async(messages)
        response = (response[response.find('{'):response.rfind('}') + 1])
        return json.loads(response)

class IntegrationAgent(Agent):
    def __init__(self, api_key: str):
        super().__init__(api_key)
        self.system_prompt = """You are an expert in integrating and validating recruitment filter solutions and converting them into specific JSON filter formats.

        Your critical responsibilities:
        1. Analyze all solutions thoroughly
        2. Provide detailed reasoning for each decision
        3. Convert the solutions into the exact filter format
        4. Only include relevant filters with valid values
        5. Ensure all values are logically consistent

        The final output must strictly follow this filter structure:
        {
            "job_title/business_function": {
                "current": [{"title_name": "", "min_staff": 0, "max_staff": 50000000}],
                "past": [{"title_name": "", "min_staff": 0, "max_staff": 50000000}],
                "both": [{"title_name": "", "min_staff": 0, "max_staff": 50000000}],
                "event": ""  # Can only be "CURRENT", "PAST", "CURRENT OR PAST", "CURRENT AND PAST"
            },
            "management_level": {
                "current": [],
                "past": [],
                "both": [],
                "event": ""
            },
            "location": {
                "current": [],
                "past": [],
                "both": [],
                "event": ""
            },
            "company": {
                "current_prompt": "",
                "past_prompt": "",
                "event": ""
            },
            "education": [{"degree": "", "major": ""}],
            "school": [],
            "current_ownership": [],
            "name": [],
            "ethnicity": [],
            "age": [],
            "total_working_years": {"min": null, "max": null},
            "role_tenure": {"min": null, "max": null},
            "company_tenure": {"min": null, "max": null},
            "skills": []
        }

        Before providing the final JSON:
        1. Analyze each requirement thoroughly
        2. Explain your reasoning for each filter value
        3. Discuss any assumptions or implicit requirements
        4. Validate logical consistency
        5. Only include filters that have valid values
        6. Explain any complex decisions or trade-offs

        Your response should follow this structure:
        1. DETAILED ANALYSIS
        2. REASONING FOR EACH FILTER
        3. ASSUMPTIONS AND IMPLICATIONS
        4. VALIDATION CHECKS
        5. FINAL FILTER JSON
        """

    def integrate(self, solutions: List[Solution], query_context: QueryContext) -> Dict:
        print("#"*47)
        print(solutions)
        print("#"*47)
        
        feedback_section = ""
        if query_context.feedback:
            feedback_section = f"""
            Previous feedback on the query solution:
            {query_context.feedback}
            
            Ensure your integration addresses this feedback.
            """
            
        prompt = f"""Integrate these solutions while maintaining alignment with the original query:

        Original Query: {query_context.original_query}

        {feedback_section}

        Please provide:
        1. Detailed analysis of how each solution component fits into the final filter structure
        2. Explicit reasoning for each filter value being set
        3. Discussion of any assumptions or implicit requirements
        4. Explanation of any trade-offs or decisions made
        5. Final filter JSON in the exact specified format

        Number of Solutions to Integrate: {len(solutions)}

        Solutions: {solutions}

        For each solution component, think through:
        - How does this map to our filter structure?
        - What are the explicit vs implicit requirements?
        - Are there any temporal aspects to consider (current vs past)?
        - What are the logical dependencies between different filters?
        - Are there any potential conflicts to resolve?

        Show your complete reasoning process before providing the final JSON structure.
        """

        messages = [
            {"role": "system", "content": self.system_prompt},
            {"role": "user", "content": prompt}
        ]
        
        response = self.call_llm(messages)
        # Split the response into reasoning and JSON parts
        parts = response.split("FINAL FILTER JSON:")
        reasoning = parts[0].strip()
        
        # Parse the JSON string
        json_str = (response[response.find('{'):response.rfind('}') + 1])
        filter_structure = json.loads(json_str)
        
        return {
            "reasoning_and_analysis": reasoning,
            "filter_structure": filter_structure
        }

# Updated Recruitment Query Processor with feedback
class FeedbackAwareRecruitmentQueryProcessor:
    def __init__(self, api_key: str):
        self.intent_agent = FeedbackAwareIntentAnalysisAgent(api_key)
        self.subproblem_agent = FeedbackAwareSubProblemAnalysisAgent(api_key)
        self.integration_agent = IntegrationAgent(api_key)

    async def process_sub_problems(self, sub_problems: List[Dict], query_context: QueryContext) -> List[Solution]:
        # Create tasks for all sub-problems
        tasks = []
        for sub_problem_dict in sub_problems:
            sub_problem = SubProblem(**sub_problem_dict)
            task = self.subproblem_agent.solve_async(sub_problem, query_context)
            tasks.append(task)
        
        # Execute all tasks concurrently
        solutions = await asyncio.gather(*tasks)
        return solutions

    async def process_query_async(self, query: str, feedback: Optional[str] = None) -> Dict:
        query_context = QueryContext(
            original_query=query,
            timestamp=datetime.now().isoformat(),
            query_id=f"query_{hash(query)}",
            feedback=feedback
        )
        
        logger.info(f"Processing query: {query_context.query_id}")
        logger.info(f"Feedback provided: {'Yes' if feedback else 'No'}")

        try:
            # Step 1: Analyze intent and decompose, incorporating feedback
            print('-'*47)
            print("Analyzing Intent and Decomposing the Problem (with feedback)...")
            analysis = self.intent_agent.analyze(query_context)
            logger.info(f"Query decomposed into {len(analysis['sub_problems'])} sub-problems")

            # Step 2: Process all sub-problems concurrently, with relevant feedback
            print('-'*47)
            print("Processing the Sub-Problems (with feedback)...")
            solutions = await self.process_sub_problems(analysis['sub_problems'], query_context)
            logger.info(f"Generated solutions for all sub-problems")

            # Step 3: Integrate solutions
            print('-'*47)
            print("Integrating Solutions of Sub-Problems...")
            final_result = self.integration_agent.integrate(solutions, query_context)
            logger.info("Solutions integrated successfully")

            print('-'*47)
            print("Printing Final Result...")
            print(json.dumps(final_result, indent=2))

            return {
                "query_context": vars(query_context),
                "analysis": analysis,
                "solutions": solutions,
                "final_result": final_result
            }

        except Exception as e:
            logger.error(f"Error processing query: {e}")
            raise

# Modified main function to demonstrate feedback
async def main():
    _ = load_dotenv()
    api_key = os.getenv("OPENAI_API_KEY")
    processor = FeedbackAwareRecruitmentQueryProcessor(api_key)
    
    queries = [
        {
            "query": "Find sales executives.",
            "feedback": "We need to focus on senior sales executives only."
        },
        # {
        #     "query": "Find VP-level leaders from companies in Western Europe with over 30 years of experience and expertise in corporate finance.",
        #     "feedback": "The results included too many people from Eastern Europe. Also, we need to see people who specifically have M&A experience."
        # },
        # {
        #     "query": "Find me a Chief Marketing Officer with a strong track record in scaling AI-driven SaaS platforms. Preferably based in the U.S.",
        #     "feedback": "The previous results included too many CMOs from consumer products. We need to focus on enterprise SaaS only. Also, we should prioritize candidates who have experience with Series B to D stage companies."
        # }
    ]
    
    query_results = []
    for query_item in queries:
        print('-'*47)
        print("QUERY: ", query_item["query"])
        print("FEEDBACK: ", query_item["feedback"])
        
        try:
            result = await processor.process_query_async(query_item["query"], query_item["feedback"])
            query_results.append({
                "query": query_item["query"], 
                "feedback": query_item["feedback"],
                "result": result
            })
        except Exception as e:
            print(f"Error processing query: {e}")
    
    return query_results

# If you just want to test a single query with feedback
async def test_single_query():
    _ = load_dotenv()
    api_key = os.getenv("OPENAI_API_KEY")
    processor = FeedbackAwareRecruitmentQueryProcessor(api_key)
    
    query = "Find VP-level leaders from AI companies in Western Europe with over 30 years of experience and expertise in corporate finance who have worked previously as Salesman."
    feedback = "The results included too many professionals from big tech companies like Google and Microsoft. We need to focus on smaller AI startups and scale-ups. Also, 'salesman' is too junior - we need people who were previously in sales management roles, not individual contributors."
    
    result = await processor.process_query_async(query, feedback)
    return result

if __name__ == "__main__":
    results = await (main())
    # Or for testing single query:
    # result = asyncio.run(test_single_query())
    
    with open("feedback_results.json", "w") as f:
        json.dump(results, f, indent=2)

INFO:__main__:Processing query: query_-3780008029369282617
INFO:__main__:Feedback provided: Yes


-----------------------------------------------
QUERY:  Find sales executives.
FEEDBACK:  We need to focus on senior sales executives only.
-----------------------------------------------
Analyzing Intent and Decomposing the Problem (with feedback)...
[
  {
    "role": "system",
    "content": "You are an expert recruitment query analyzer specializing in breaking down complex queries into logical sub-problems while incorporating feedback.\n\n        Your primary responsibilities:\n        1. Deeply understand the main intent of the recruitment query and the feedback\n        2. Analyze and incorporate any provided feedback\n        3. Break down the query into distinct, manageable sub-problems according to the provided feedback\n        4. Identify and maintain relationships between sub-problems based on the feedback\n        5. Preserve context and references to the original query and feedback\n        6. Assign priority levels (1-5, 1 being highest)\n        7. Connect relevant parts

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:__main__:Query decomposed into 2 sub-problems


```json
{
    "main_intent": "Identify senior sales executives for recruitment",
    "feedback_analysis": "Feedback emphasizes the need to focus on senior-level roles rather than general sales executives.",
    "thinking": "The core requirement is to find sales executives, but feedback narrows this down to specifically target senior sales executives. Therefore, the focus should shift from broad sales executive roles to those with seniority.",
    "sub_problems": [
        {
            "category": "job_title",
            "description": "Ensure candidates are identified as senior sales executives.",
            "dependencies": [],
            "priority": 1,
            "context_reference": {"query_segment": "Find sales executives"},
            "query_segment": "sales executives",
            "relevant_feedback": "Feedback specifies the need to focus on senior sales executives only."
        },
        {
            "category": "experience_level",
            "description": "Determine 

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:__main__:Generated solutions for all sub-problems


-----------------------------------------------
Integrating Solutions of Sub-Problems...
###############################################
[{'sub_problem_analysis': {'original_context': 'The original query context is to find sales executives, with a specific focus on identifying senior sales executives as per the feedback.', 'feedback_integration': 'The feedback emphasizes the importance of refining the search to focus specifically on senior sales executives, rather than just any sales executive.', 'interpreted_requirement': "The requirement is to identify and filter candidates who specifically hold the job title of 'Senior Sales Executive' or similar senior-level sales roles.", 'thought_process': "To address the feedback, we need to ensure that the filtering mechanism can distinguish between general sales executives and those at a senior level. This involves considering job title variations that imply seniority, such as 'Senior Sales Executive', 'Lead Sales Executive', or 'Sales Manager

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:__main__:Solutions integrated successfully


1. **DETAILED ANALYSIS**

The task is to integrate two solutions to refine the search for senior sales executives. The solutions present a multi-faceted approach to defining seniority, focusing on job titles, years of experience, leadership roles, and achievements. The feedback emphasizes a need to focus specifically on senior sales executives, not just any sales executives.

2. **REASONING FOR EACH FILTER**

- **Job Title/Business Function**: The first solution focuses on specific job titles that indicate seniority, like "Senior Sales Executive," "Lead Sales Executive," and "Sales Manager." This directly maps to the "job_title/business_function" filter. By including these titles, we align with the feedback to target senior roles explicitly.

- **Total Working Years**: The second solution suggests using years of experience as a seniority marker, proposing a threshold of 10+ years. This maps to the "total_working_years" filter, ensuring candidates have significant work experience.

- **

In [ ]:
from typing import Dict, List, Any, Optional
from dataclasses import dataclass
import openai
import json
from enum import Enum
from datetime import datetime
import logging
from dotenv import load_dotenv
import os
from openai import OpenAI
import ast
from pprint import pprint
from openai import AsyncOpenAI
import asyncio
import traceback

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

@dataclass
class QueryContext:
    original_query: str
    timestamp: str
    query_id: str
    feedback: Optional[str] = None
    filter_values: Optional[Dict[str, Any]] = None  # Added filter_values field
    
@dataclass
class SubProblem:
    category: str
    description: str
    dependencies: List[str]
    priority: int
    context_reference: Dict[str, Any]
    query_segment: str
    relevant_feedback: Optional[str] = None
    relevant_filters: Optional[Dict[str, Any]] = None  # Added relevant_filters field

@dataclass
class Solution:
    sub_problem: SubProblem
    proposed_solution: Dict
    confidence: float
    reasoning: str
    validation_notes: List[str]

class Agent:
    def __init__(self, api_key: str):
        self.client = OpenAI(api_key=api_key)
        self.async_client = AsyncOpenAI(api_key=api_key)
        
    def call_llm(self, messages: List[Dict]) -> str:
        try:
            print(json.dumps(messages, indent=2))
            response = self.client.chat.completions.create(
                model="gpt-4o",
                messages=messages,
                temperature=0.7
            )
            print((response.choices[0].message.content))
            return response.choices[0].message.content
        except Exception as e:
            logger.error(f"Error in LLM call: {e}")
            raise

    async def call_llm_async(self, messages: List[Dict]) -> str:
        try:
            response = await self.async_client.chat.completions.create(
                model="gpt-4o",
                messages=messages,
                temperature=0.7
            )
            return response.choices[0].message.content
        except Exception as e:
            logger.error(f"Error in async LLM call: {e}")
            raise

class FilterAwareIntentAnalysisAgent(Agent):
    def __init__(self, api_key: str):
        super().__init__(api_key)
        self.system_prompt = """You are an expert recruitment query analyzer specializing in breaking down complex queries into logical sub-problems while incorporating feedback and existing filter values.

        Your primary responsibilities:
        1. Deeply understand the main intent of the recruitment query, the feedback, and any existing filter values
        2. Analyze and incorporate any provided feedback
        3. Consider existing filter values when breaking down the query
        4. Break down the query into distinct, manageable sub-problems
        5. Identify and maintain relationships between sub-problems
        6. Preserve context and references to the original query, feedback, and filter values
        7. Assign priority levels (1-5, 1 being highest)
        8. Connect relevant parts of the feedback to specific sub-problems
        9. Assign relevant filter values to each sub-problem
        10. While breaking down the query into sub-problems, your main focus should be to focus on the provided feedback and existing filter values.
        
        Think through these steps:
        1. What is the core requirement?
        2. What are the mandatory vs optional criteria?
        3. Are there implicit requirements that need to be made explicit?
        4. How do different parts of the query relate to each other?
        5. What potential ambiguities need to be resolved?
        6. How does the feedback modify, clarify, or redirect the original query?
        7. Which specific feedback points apply to which sub-problems?
        8. How do existing filter values constrain or guide the solution?
        9. Which filter values are relevant to which sub-problems?

        For each sub-problem, consider:
        - Which part of the original query does it address?
        - What other sub-problems might it depend on?
        - What specific filter categories does it relate to?
        - What potential challenges might arise in solving it?
        - What specific feedback points are relevant to this sub-problem?
        - What specific filter values are relevant to this sub-problem?
        - How should the feedback and filter values influence this sub-problem's solution?

        Output JSON structure:
        {
            "main_intent": "Clear statement of primary goal",
            "feedback_analysis": "Analysis of how feedback modifies the query",
            "filter_analysis": "Analysis of how existing filter values constrain the solution",
            "thinking" : "",
            "sub_problems": [
                {
                    "category": "job_title/location/education/etc",
                    "description": "Detailed description",
                    "dependencies": ["other_category_ids"],
                    "priority": "1-5",
                    "context_reference": {"query_segment": "relevant part of original query"},
                    "query_segment": "exact text from original query",
                    "relevant_feedback": "feedback specifically relevant to this sub-problem",
                    "relevant_filters": {"filter_name": "filter_value"} 
                }
            ],
            "analysis_notes": ["Detailed thoughts and reasoning"],
            "potential_challenges": ["Identified challenges"],
            "confidence_score": 0.0
        }"""

    def analyze(self, query_context: QueryContext) -> Dict:
        feedback_section = ""
        if query_context.feedback:
            feedback_section = f"""
            The following feedback has been provided about previous attempts to solve this query:
            {query_context.feedback}
            
            When analyzing the query, carefully incorporate this feedback to improve the decomposition.
            For each sub-problem, identify specific parts of the feedback that apply to it.
            """
        
        filter_section = ""
        if query_context.filter_values:
            filter_section = f"""
            The following filter values already exist from previous processing:
            {json.dumps(query_context.filter_values, indent=2)}
            
            Consider these filter values when breaking down the query into sub-problems.
            Identify which filter values are relevant to each sub-problem.
            """
        
        prompt = f"""Analyze this recruitment query with extreme attention to detail: {query_context.original_query}

        {feedback_section}
        
        {filter_section}

        Consider these specific aspects:
        1. What are the explicit requirements?
        2. What are the implicit requirements?
        3. Are there any ambiguous terms that need clarification?
        4. How do different requirements relate to each other?
        5. How does the feedback modify our understanding of the requirements?
        6. How do existing filter values constrain or guide our solution?
        
        Show your reasoning process."""

        messages = [
            {"role": "system", "content": self.system_prompt},
            {"role": "user", "content": prompt}
        ]
        
        response = self.call_llm(messages)
        response = eval(response[response.find('{'):response.rfind('}') + 1])
        return response

class FilterAwareSubProblemAnalysisAgent(Agent):
    def __init__(self, api_key: str):
        super().__init__(api_key)
        self.system_prompt = """You are an expert in solving specific recruitment sub-problems, with the ability to incorporate targeted feedback and existing filter values.
        IMPORTANT: Focus ONLY on the specific sub-problem assigned to you. While you may reference
        the original query for context, your solution should be specifically targeted to your assigned sub-problem.

        For this specific sub-problem, follow this detailed analysis process:
        1. Initial Assessment
           - Understand the specific requirement of THIS sub-problem
           - Review any specific feedback provided for this sub-problem
           - Review any existing filter values relevant to this sub-problem
           - Identify relevant filter categories for THIS component
           - Review dependencies and constraints specific to THIS aspect

        2. Feedback and Filter Integration
           - Carefully analyze how the provided feedback applies to this specific sub-problem
           - Consider how existing filter values constrain or guide your solution
           - Adjust your approach based on the feedback and filter values
           - Address any issues or improvements mentioned in the feedback
           - Build upon or refine existing filter values

        3. Solution Generation
           - Generate multiple potential approaches for THIS specific requirement
           - Consider edge cases and limitations
           - Ensure solution stays focused on the assigned sub-problem
           - Solution should be so that it is only to generate a specific filter value
           - Apply the lessons learned from feedback
           - Consider compatibility with existing filter values
        
        Note: While you are generating proposed solutions, you need to think mostly in terms of using which filter and what possible values can be used for that filter.

        4. Solution Evaluation
           - Score each solution (1-10)
           - List pros and cons specific to this sub-problem
           - Consider implementation feasibility
           - Evaluate how well the solution addresses the feedback
           - Evaluate how well the solution integrates with existing filter values

        5. Final Selection
           - Choose optimal solution for this specific component
           - Document reasoning
           - Provide confidence score
           - Explain how feedback influenced your final choice
           - Explain how existing filter values influenced your final choice

        Remember: Stay focused on your assigned sub-problem. Don't try to solve the entire query.

        Output structure:
        {
            "sub_problem_analysis": {
                "original_context": "",
                "feedback_integration": "",
                "filter_integration": "",
                "interpreted_requirement": "",
                "thought_process": "",
                "proposed_solutions": [
                    {
                        "thinking" : "",
                        "approach": "",
                        "score": 0,
                        "pros": [],
                        "cons": [],
                        "implementation_notes": "",
                        "feedback_addressed": "",
                        "filter_compatibility": ""
                    }
                ],
                "selected_solution": {
                    "thinking_for_selection" : "",
                    "filter_values": {},
                    "confidence": 0.0,
                    "reasoning": "",
                    "validation_notes": [],
                    "feedback_influence": "",
                    "filter_influence": ""
                }
            }
        }"""

    async def solve_async(self, sub_problem: SubProblem, query_context: QueryContext) -> Solution:
        feedback_section = ""
        if sub_problem.relevant_feedback:
            feedback_section = f"""
            The following feedback is specifically relevant to this sub-problem:
            {sub_problem.relevant_feedback}
            
            Make sure your solution directly addresses this feedback.
            """
        elif query_context.feedback:
            feedback_section = f"""
            General feedback on the query (consider only what's relevant to this sub-problem):
            {query_context.feedback}
            """

        filter_section = ""
        if sub_problem.relevant_filters:
            filter_section = f"""
            The following filter values are specifically relevant to this sub-problem:
            {json.dumps(sub_problem.relevant_filters, indent=2)}
            
            Consider these filter values when developing your solution.
            You can build upon, refine, or modify these filter values as needed.
            """
        elif query_context.filter_values:
            filter_section = f"""
            General filter values from previous processing (consider only what's relevant to this sub-problem):
            {json.dumps(query_context.filter_values, indent=2)}
            """

        prompt = f"""Focus on solving this specific sub-problem:

        Sub-problem Category: {sub_problem.category}
        Sub-problem Description: {sub_problem.description}
        Specific Query Segment to Address: {sub_problem.query_segment}

        {feedback_section}
        
        {filter_section}

        For reference only, original query context: {query_context.original_query}

        Remember: Focus only on solving this specific sub-problem component.
        Do not attempt to solve the entire query.

        Show your detailed reasoning process for this specific component,
        including how you've incorporated the relevant feedback and filter values."""

        messages = [
            {"role": "system", "content": self.system_prompt},
            {"role": "user", "content": prompt}
        ]
        
        response = await self.call_llm_async(messages)
        response = (response[response.find('{'):response.rfind('}') + 1])
        return json.loads(response)

class IntegrationAgent(Agent):
    def __init__(self, api_key: str):
        super().__init__(api_key)
        self.system_prompt = """You are an expert in integrating and validating recruitment filter solutions and converting them into specific JSON filter formats.

        Your critical responsibilities:
        1. Analyze all solutions thoroughly
        2. Provide detailed reasoning for each decision
        3. Convert the solutions into the exact filter format
        4. Only include relevant filters with valid values
        5. Ensure all values are logically consistent
        6. Consider feedback and existing filter values in your integration

        The final output must strictly follow this filter structure:
        {
            "job_title/business_function": {
                "current": [{"title_name": "", "min_staff": 0, "max_staff": 50000000}],
                "past": [{"title_name": "", "min_staff": 0, "max_staff": 50000000}],
                "both": [{"title_name": "", "min_staff": 0, "max_staff": 50000000}],
                "event": ""  # Can only be "CURRENT", "PAST", "CURRENT OR PAST", "CURRENT AND PAST"
            },
            "management_level": {
                "current": [],
                "past": [],
                "both": [],
                "event": ""
            },
            "location": {
                "current": [],
                "past": [],
                "both": [],
                "event": ""
            },
            "company": {
                "current_prompt": "",
                "past_prompt": "",
                "event": ""
            },
            "education": [{"degree": "", "major": ""}],
            "school": [],
            "current_ownership": [],
            "name": [],
            "ethnicity": [],
            "age": [],
            "total_working_years": {"min": null, "max": null},
            "role_tenure": {"min": null, "max": null},
            "company_tenure": {"min": null, "max": null},
            "skills": []
        }

        Before providing the final JSON:
        1. Analyze each requirement thoroughly
        2. Explain your reasoning for each filter value
        3. Discuss any assumptions or implicit requirements
        4. Validate logical consistency
        5. Only include filters that have valid values
        6. Explain any complex decisions or trade-offs
        7. Discuss how feedback has been addressed
        8. Explain how existing filter values have been incorporated

        Your response should follow this structure:
        1. DETAILED ANALYSIS
        2. REASONING FOR EACH FILTER
        3. ASSUMPTIONS AND IMPLICATIONS
        4. VALIDATION CHECKS
        5. FEEDBACK INCORPORATION
        6. FINAL FILTER JSON
        """

    def integrate(self, solutions: List[Solution], query_context: QueryContext) -> Dict:
        print("#"*47)
        print(solutions)
        print("#"*47)
        
        feedback_section = ""
        if query_context.feedback:
            feedback_section = f"""
            Previous feedback on the query solution:
            {query_context.feedback}
            
            Ensure your integration addresses this feedback.
            """
            
        filter_section = ""
        if query_context.filter_values:
            filter_section = f"""
            Existing filter values from previous processing:
            {json.dumps(query_context.filter_values, indent=2)}
            
            Consider these filter values in your integration.
            You can build upon, refine, or modify these filter values as needed.
            """
            
        prompt = f"""Integrate these solutions while maintaining alignment with the original query:

        Original Query: {query_context.original_query}

        {feedback_section}
        
        {filter_section}

        Please provide:
        1. Detailed analysis of how each solution component fits into the final filter structure
        2. Explicit reasoning for each filter value being set
        3. Discussion of any assumptions or implicit requirements
        4. Explanation of any trade-offs or decisions made
        5. Explanation of how feedback has been addressed
        6. Explanation of how existing filter values have been incorporated
        7. Final filter JSON in the exact specified format

        Number of Solutions to Integrate: {len(solutions)}

        Solutions: {solutions}

        For each solution component, think through:
        - How does this map to our filter structure?
        - What are the explicit vs implicit requirements?
        - Are there any temporal aspects to consider (current vs past)?
        - What are the logical dependencies between different filters?
        - Are there any potential conflicts to resolve?
        - How does the feedback influence this component?
        - How do existing filter values influence this component?

        Show your complete reasoning process before providing the final JSON structure.
        """

        messages = [
            {"role": "system", "content": self.system_prompt},
            {"role": "user", "content": prompt}
        ]
        
        response = self.call_llm(messages)
        # Split the response into reasoning and JSON parts
        parts = response.split("FINAL FILTER JSON:")
        reasoning = parts[0].strip()
        
        # Parse the JSON string
        json_str = (response[response.find('{'):response.rfind('}') + 1])
        filter_structure = json.loads(json_str)
        
        return {
            "reasoning_and_analysis": reasoning,
            "filter_structure": filter_structure
        }

# Updated Recruitment Query Processor with filter values
class FilterAwareRecruitmentQueryProcessor:
    def __init__(self, api_key: str):
        self.intent_agent = FilterAwareIntentAnalysisAgent(api_key)
        self.subproblem_agent = FilterAwareSubProblemAnalysisAgent(api_key)
        self.integration_agent = IntegrationAgent(api_key)

    async def process_sub_problems(self, sub_problems: List[Dict], query_context: QueryContext) -> List[Solution]:
        # Create tasks for all sub-problems
        tasks = []
        for sub_problem_dict in sub_problems:
            sub_problem = SubProblem(**sub_problem_dict)
            task = self.subproblem_agent.solve_async(sub_problem, query_context)
            tasks.append(task)
        
        # Execute all tasks concurrently
        solutions = await asyncio.gather(*tasks)
        return solutions

    async def process_query_async(self, query: str, feedback: Optional[str] = None, filter_values: Optional[Dict[str, Any]] = None) -> Dict:
        query_context = QueryContext(
            original_query=query,
            timestamp=datetime.now().isoformat(),
            query_id=f"query_{hash(query)}",
            feedback=feedback,
            filter_values=filter_values
        )
        
        logger.info(f"Processing query: {query_context.query_id}")
        logger.info(f"Feedback provided: {'Yes' if feedback else 'No'}")
        logger.info(f"Filter values provided: {'Yes' if filter_values else 'No'}")

        try:
            # Step 1: Analyze intent and decompose, incorporating feedback and filter values
            print('-'*47)
            print("Analyzing Intent and Decomposing the Problem (with feedback and filter values)...")
            analysis = self.intent_agent.analyze(query_context)
            logger.info(f"Query decomposed into {len(analysis['sub_problems'])} sub-problems")

            # Step 2: Process all sub-problems concurrently, with relevant feedback and filter values
            print('-'*47)
            print("Processing the Sub-Problems (with feedback and filter values)...")
            solutions = await self.process_sub_problems(analysis['sub_problems'], query_context)
            logger.info(f"Generated solutions for all sub-problems")

            # Step 3: Integrate solutions
            print('-'*47)
            print("Integrating Solutions of Sub-Problems...")
            final_result = self.integration_agent.integrate(solutions, query_context)
            logger.info("Solutions integrated successfully")

            print('-'*47)
            print("Printing Final Result...")
            print(json.dumps(final_result, indent=2))

            return {
                "query_context": vars(query_context),
                "analysis": analysis,
                "solutions": solutions,
                "final_result": final_result
            }

        except Exception as e:
            logger.error(f"Error processing query: {e}")
            raise

# Modified main function to demonstrate feedback and filter values
async def main():
    _ = load_dotenv()
    api_key = os.getenv("OPENAI_API_KEY")
    processor = FilterAwareRecruitmentQueryProcessor(api_key)
    
    # Example of filter values from a previous run
    example_filter_values = {
        "job_title/business_function": {
            "current": [{"title_name": "Sales Executive", "min_staff": 0, "max_staff": 50000000}],
            "past": [],
            "both": [],
            "event": "CURRENT"
        },
        "management_level": {
            "current": [],
            "past": [],
            "both": [],
            "event": ""
        },
        "location": {
            "current": [],
            "past": [],
            "both": [],
            "event": ""
        }
    }
    
    queries = [
        # {
        #     "query": "Find sales executives.",
        #     "feedback": "We need to focus on senior sales executives only.",
        #     "filter_values": example_filter_values
        # },
        # {
        #     "query": "Find VP-level leaders from companies in Western Europe with over 30 years of experience and expertise in corporate finance.",
        #     "feedback": "The results included too many people from Eastern Europe. Also, we need to see people who specifically have M&A experience.",
        #     "filter_values": None
        # },
        {
            "query": "Find me a Chief Marketing Officer with a strong track record in scaling AI-driven SaaS platforms. Preferably based in the U.S.",
            "feedback": "People who are not CMOs are appearing a lot. The results included too many people from US. Also, we need to see people who specifically have worked in sales in the past.",
            "filter_values": """{
    "jobTitles": {
        "innerLogic": "OR",
        "current": {
            "isStrict": false,
            "isRemoved": [],
            "isSelected": [
                {
                    "max": 50000000,
                    "min": 1,
                    "name": "CMO"
                },
                {
                    "max": 50000000,
                    "min": 1,
                    "name": "Chief Marketing Officer"
                },
                {
                    "max": 50000000,
                    "min": 1,
                    "name": "Director of Marketing"
                },
                {
                    "max": 50000000,
                    "min": 1,
                    "name": "Marketing Executive"
                },
                {
                    "max": 50000000,
                    "min": 1,
                    "name": "Vice President of Marketing"
                },
                {
                    "max": 50000000,
                    "min": 1,
                    "name": "Head of Marketing"
                },
                {
                    "max": 50000000,
                    "min": 1,
                    "name": "Marketing Director"
                },
                {
                    "max": 50000000,
                    "min": 1,
                    "name": "Senior Vice President of Marketing"
                },
                {
                    "max": 50000000,
                    "min": 1,
                    "name": "Senior Marketing Director"
                }
            ]
        },
        "previous": {
            "isStrict": false,
            "isRemoved": [],
            "isSelected": []
        }
    },
    "company": {
        "innerLogic": "OR",
        "current": {
            "isStrict": false,
            "isRemoved": [],
            "isSelected": []
        },
        "previous": {
            "isStrict": false,
            "isRemoved": [],
            "isSelected": []
        }
    },
    "locations": {
        "innerLogic": "OR",
        "current": {
            "isStrict": false,
            "isRemoved": [],
            "isSelected": [
                {
                    "id": 103644278,
                    "urn": "urn:li:fs_geo:103644278",
                    "name": "United States"
                }
            ]
        },
        "previous": {
            "isStrict": false,
            "isRemoved": [],
            "isSelected": []
        }
    },
    "managementTitles": {
        "innerLogic": "OR",
        "current": {
            "isStrict": false,
            "isRemoved": [],
            "isSelected": [
                {
                    "name": "C Suite"
                },
                {
                    "name": "Executive and Sr. VP"
                },
                {
                    "name": "VP"
                },
                {
                    "name": "Head"
                },
                {
                    "name": "Director"
                },
                {
                    "name": "President"
                }
            ]
        },
        "previous": {
            "isStrict": false,
            "isRemoved": [],
            "isSelected": []
        }
    },
    "education": {
        "innerLogic": "OR",
        "current": {
            "isStrict": false,
            "isRemoved": [],
            "isSelected": []
        },
        "previous": {
            "isStrict": false,
            "isRemoved": [],
            "isSelected": []
        }
    },
    "companyOwnershipTitles": {
        "innerLogic": "OR",
        "current": {
            "isStrict": false,
            "isRemoved": [],
            "isSelected": []
        },
        "previous": {
            "isStrict": false,
            "isRemoved": [],
            "isSelected": []
        }
    },
    "university": {
        "innerLogic": "OR",
        "current": {
            "isStrict": false,
            "isRemoved": [],
            "isSelected": []
        },
        "previous": {
            "isStrict": false,
            "isRemoved": [],
            "isSelected": []
        }
    },
    "industries": {
        "innerLogic": "OR",
        "current": {
            "isStrict": false,
            "isRemoved": [],
            "isSelected": [
                {
                    "name": "Enterprise SaaS/Software"
                },
                {
                    "name": "Artificial Intelligence/Machine Learning Companies"
                },
                {
                    "name": "Marketing Technology"
                },
                {
                    "name": "Cloud Computing/Services"
                }
            ]
        },
        "previous": {
            "isStrict": false,
            "isRemoved": [],
            "isSelected": []
        }
    },
    "skills": {
        "innerLogic": "OR",
        "current": {
            "isStrict": false,
            "isRemoved": [],
            "isSelected": []
        },
        "previous": {
            "isStrict": false,
            "isRemoved": [],
            "isSelected": []
        },
        "either": {
            "isStrict": false,
            "isRemoved": [],
            "isSelected": [
                {
                    "name": "SaaS marketing"
                },
                {
                    "name": "AI/ML product marketing"
                },
                {
                    "name": "Marketing automation"
                },
                {
                    "name": "Growth marketing"
                },
                {
                    "name": "B2B technology marketing"
                },
                {
                    "name": "Scaling Teams"
                },
                {
                    "name": "Growth Strategy"
                }
            ]
        }
    },
    "product": {
        "innerLogic": "OR",
        "current": {
            "isStrict": false,
            "isRemoved": [],
            "isSelected": []
        },
        "previous": {
            "isStrict": false,
            "isRemoved": [],
            "isSelected": []
        }
    },
    "profileNames": {
        "isSelected": [],
        "isRemoved": []
    },
    "experience": {
        "total": {
            "max": 100,
            "min": 5
        }
    },
    "roleTenure": {
        "total": {
            "min": 1,
            "max": 100
        }
    },
    "companyTenure": {
        "total": {
            "max": 100,
            "min": 3
        }
    },
    "demographic": {
        "innerLogic": "OR",
        "current": {
            "isStrict": false,
            "isRemoved": [],
            "isSelected": []
        },
        "previous": {
            "isStrict": false,
            "isRemoved": [],
            "isSelected": []
        }
    },
    "isCompanyTenureSelected": true,
    "isRoleTenureSelected": false,
    "isExperienceSelected": true,
    "isCompanyOwnershipSelected": false
}"""
        },
    ]
    
    query_results = []
    for query_item in queries:
        print('-'*47)
        print("QUERY: ", query_item["query"])
        print("FEEDBACK: ", query_item["feedback"])
        print("FILTER VALUES: ", "Provided" if query_item["filter_values"] else "None")
        
        try:
            result = await processor.process_query_async(
                query_item["query"], 
                query_item["feedback"],
                query_item["filter_values"]
            )
            query_results.append({
                "query": query_item["query"], 
                "feedback": query_item["feedback"],
                "filter_values": query_item["filter_values"],
                "result": result
            })
        except Exception as e:
            traceback.print_exc()
            print(f"Error processing query: {e}")
    
    return query_results

# If you just want to test a single query with feedback and filter values
async def test_single_query():
    _ = load_dotenv()
    api_key = os.getenv("OPENAI_API_KEY")
    processor = FilterAwareRecruitmentQueryProcessor(api_key)
    
    query = "Find VP-level leaders from AI companies in Western Europe with over 30 years of experience and expertise in corporate finance who have worked previously as Salesman."
    feedback = "The results included too many professionals from big tech companies like Google and Microsoft. We need to focus on smaller AI startups and scale-ups. Also, 'salesman' is too junior - we need people who were previously in sales management roles, not individual contributors."
    
    # Example filter values
    filter_values = {
        "job_title/business_function": {
            "current": [{"title_name": "VP", "min_staff": 0, "max_staff": 50000000}],
            "past": [{"title_name": "Salesman", "min_staff": 0, "max_staff": 50000000}],
            "both": [],
            "event": "CURRENT AND PAST"
        },
        "location": {
            "current": [{"continent": "Europe", "region": "Western Europe"}],
            "past": [],
            "both": [],
            "event": "CURRENT"
        },
        "total_working_years": {"min": 30, "max": null}
    }
    
    result = await processor.process_query_async(query, feedback, filter_values)
    return result

if __name__ == "__main__":
    results = await (main())
    # Or for testing single query:
    # result = asyncio.run(test_single_query())
    
    with open("filter_feedback_results.json", "w") as f:
        json.dump(results, f, indent=2)

INFO:__main__:Processing query: query_-530054364588539445
INFO:__main__:Feedback provided: Yes
INFO:__main__:Filter values provided: Yes


-----------------------------------------------
QUERY:  Find me a Chief Marketing Officer with a strong track record in scaling AI-driven SaaS platforms. Preferably based in the U.S.
FEEDBACK:  People who are not CMOs are appearing a lot. The results included too many people from US. Also, we need to see people who specifically have worked in sales in the past.
FILTER VALUES:  Provided
-----------------------------------------------
Analyzing Intent and Decomposing the Problem (with feedback and filter values)...
[
  {
    "role": "system",
    "content": "You are an expert recruitment query analyzer specializing in breaking down complex queries into logical sub-problems while incorporating feedback and existing filter values.\n\n        Your primary responsibilities:\n        1. Deeply understand the main intent of the recruitment query, the feedback, and any existing filter values\n        2. Analyze and incorporate any provided feedback\n        3. Consider existing filter values wh

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:__main__:Query decomposed into 4 sub-problems


```json
{
    "main_intent": "Identify a Chief Marketing Officer with a strong track record in scaling AI-driven SaaS platforms, preferably based in the U.S., with a background in sales.",
    "feedback_analysis": "The feedback indicates that the current query processing is inadvertently including individuals who are not CMOs and has an over-representation of candidates from the US. Additionally, there is a need to ensure candidates have a past experience in sales.",
    "filter_analysis": "Existing filters are set to include a variety of senior marketing roles, broad geographic inclusion of the U.S., and specific skills and industries related to SaaS and AI. These filters need to be refined to address feedback about job title specificity and geographic distribution.",
    "thinking": "The core requirement is to find a CMO with experience in scaling AI-driven SaaS platforms, who preferably is located in the U.S. The feedback emphasizes the need to be more precise in filtering for CMOs 

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:__main__:Generated solutions for all sub-problems


-----------------------------------------------
Integrating Solutions of Sub-Problems...
###############################################
[{'sub_problem_analysis': {'original_context': 'Find me a Chief Marketing Officer with a strong track record in scaling AI-driven SaaS platforms. Preferably based in the U.S.', 'feedback_integration': 'The feedback indicates that people who are not CMOs are appearing frequently, which suggests that the current filters are too broad or not specific enough to capture only those who are at the CMO level or equivalent.', 'filter_integration': "Currently existing filter values include job titles like 'CMO' and 'Chief Marketing Officer', as well as management titles such as 'C Suite', 'Executive and Sr. VP', and 'VP'. These filters need to be refined to more accurately target CMOs or equivalent senior marketing roles.", 'interpreted_requirement': 'The requirement is to ensure that only candidates who hold the title of Chief Marketing Officer or an equivalen

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:__main__:Solutions integrated successfully


### 1. DETAILED ANALYSIS

#### Solution Components Mapping to Filter Structure:

1. **Job Title/Business Function**:
    - The requirement is to filter candidates with titles such as "Chief Marketing Officer" or equivalent roles. The feedback suggests refining the list to avoid non-CMO roles.
    - Include past experience in sales roles, which indicates a need for integration of past roles related to sales.
    - Temporal aspect: Separate current and past roles to distinguish between their current role as CMO and past sales experience.

2. **Management Level**:
    - Focus on C-suite or equivalent senior roles, which aligns with the feedback to exclude broader titles like "VP" unless they are directly related to marketing.

3. **Location**:
    - Original focus was on U.S.-based candidates. Feedback suggests broadening this to include other major English-speaking countries and relevant tech hubs to ensure diversity.

4. **Skills**:
    - Skills related to scaling SaaS platforms and AI-

In [11]:
from typing import Dict, List, Any, Optional
from dataclasses import dataclass
import openai
import json
from enum import Enum
from datetime import datetime
import logging
from dotenv import load_dotenv
import os
from openai import OpenAI
import ast
from pprint import pprint
from openai import AsyncOpenAI
import asyncio
import traceback

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

@dataclass
class QueryContext:
    original_query: str
    timestamp: str
    query_id: str
    feedback: Optional[str] = None
    filter_values: Optional[Dict[str, Any]] = None  # Added filter_values field
    
@dataclass
class SubProblem:
    category: str
    description: str
    dependencies: List[str]
    priority: int
    context_reference: Dict[str, Any]
    query_segment: str
    relevant_feedback: Optional[str] = None
    relevant_filters: Optional[Dict[str, Any]] = None  # Added relevant_filters field

@dataclass
class Solution:
    sub_problem: SubProblem
    proposed_solution: Dict
    confidence: float
    reasoning: str
    validation_notes: List[str]

class Agent:
    def __init__(self, api_key: str):
        self.client = OpenAI(api_key=api_key)
        self.async_client = AsyncOpenAI(api_key=api_key)
        
    def call_llm(self, messages: List[Dict]) -> str:
        try:
            print(json.dumps(messages, indent=2))
            response = self.client.chat.completions.create(
                model="gpt-4o",
                messages=messages,
                temperature=0.7
            )
            print((response.choices[0].message.content))
            return response.choices[0].message.content
        except Exception as e:
            logger.error(f"Error in LLM call: {e}")
            raise

    async def call_llm_async(self, messages: List[Dict]) -> str:
        try:
            response = await self.async_client.chat.completions.create(
                model="gpt-4o",
                messages=messages,
                temperature=0.7
            )
            return response.choices[0].message.content
        except Exception as e:
            logger.error(f"Error in async LLM call: {e}")
            raise


class FilterAwareIntentAnalysisAgent(Agent):
    def __init__(self, api_key: str):
        super().__init__(api_key)
        self.system_prompt = """You are an expert recruitment query analyzer specializing in breaking down complex queries into ATOMIC logical sub-problems while prioritizing feedback incorporation and considering existing filter values.

        Your PRIMARY RESPONSIBILITIES (in order of importance):
        1. FEEDBACK FOCUS: Deeply analyze any provided feedback and make it the central guiding principle of your analysis
        2. ATOMICITY: Break down queries into truly atomic sub-problems (each addressing ONE specific aspect only)
        3. FILTER AWARENESS: Carefully consider existing filter values when analyzing the query
        4. INTENT UNDERSTANDING: Comprehend the main intent of the recruitment query
        
        Key principles for FEEDBACK incorporation:
        - Treat feedback as the highest priority guidance for your analysis
        - Explicitly connect each piece of feedback to specific atomic sub-problems
        - If feedback contradicts the original query, prioritize the feedback's direction
        - Use feedback to refine, modify, or completely change your understanding of requirements
        - For each atomic sub-problem, include SPECIFIC quotes from the feedback that are relevant
        
        Key principles for ATOMIC SUB-PROBLEM creation:
        - Each sub-problem must address EXACTLY ONE specific requirement or concept
        - If a potential sub-problem addresses multiple requirements, split it further
        - Test each sub-problem with the question: "Does this solve exactly one thing?"
        - Avoid compound requirements within a single sub-problem
        - Ensure each sub-problem has a singular, clear focus
        
        For each ATOMIC sub-problem, you must:
        1. Focus on a single, specific aspect of the requirement
        2. Connect relevant feedback specifically to this atomic element
        3. Identify which specific filter values (if any) relate to this single element
        4. Assign appropriate priority (1-5, 1 being highest)
        5. Explicitly state dependencies on other atomic sub-problems
        
        Your analysis process should follow these steps:
        1. First, thoroughly analyze the feedback to identify key guidance points
        2. Second, examine existing filter values to understand constraints
        3. Third, break down the query into the smallest possible atomic sub-problems
        4. Finally, connect feedback points and filter values to each atomic sub-problem
        
        Output JSON structure:
        {
            "main_intent": "Clear statement of primary goal",
            "feedback_analysis": "COMPREHENSIVE analysis of how feedback modifies the query - this should be DETAILED",
            "filter_analysis": "Analysis of how existing filter values constrain the solution",
            "thinking": "Step-by-step breakdown of how you arrived at your atomic sub-problems",
            "analysis_notes": ["Detailed thoughts and reasoning"],
            "potential_challenges": ["Identified challenges"],
            "sub_problems": [
                {
                    "category": "job_title/location/education/etc",
                    "description": "Description of this SINGLE atomic requirement",
                    "dependencies": ["other_category_ids"],
                    "priority": "1-5",
                    "context_reference": {"query_segment": "relevant part of original query"},
                    "query_segment": "exact text from original query",
                    "relevant_feedback": "SPECIFIC feedback directly relevant to this atomic sub-problem",
                    "relevant_filters": {"filter_name": "filter_value"} 
                }
            ],
            "confidence_score": 0.0
        }"""

    def analyze(self, query_context: QueryContext) -> Dict:
        feedback_section = ""
        if query_context.feedback:
            feedback_section = f"""
            CRITICAL FEEDBACK INFORMATION:
            The following feedback has been provided about previous attempts to solve this query:
            {query_context.feedback}
            
            THIS FEEDBACK IS YOUR PRIMARY GUIDANCE. You must:
            1. Make this feedback the central focus of your analysis
            2. For each atomic sub-problem, identify specific parts of the feedback that apply to it
            3. Use feedback to reinterpret or modify the original query when needed
            4. If feedback conflicts with the original query, prioritize the feedback's direction
            5. Extract specific quotes from the feedback for each atomic sub-problem
            """
        
        filter_section = ""
        if query_context.filter_values:
            filter_section = f"""
            EXISTING FILTER VALUES:
            {json.dumps(query_context.filter_values, indent=2)}
            
            Consider these filter values when creating atomic sub-problems:
            1. For each atomic sub-problem, identify specific relevant filter values
            2. Understand how each filter constrains or guides that atomic requirement
            3. Note where existing filters might already address parts of the query
            """
        
        prompt = f"""Analyze this recruitment query, focusing on breaking it down into TRULY ATOMIC sub-problems: 
        {query_context.original_query}

        {feedback_section}
        
        {filter_section}

        IMPORTANT ANALYSIS INSTRUCTIONS:
        1. PRIMARILY focus on the feedback (if provided) to guide your understanding
        2. Break the query into the SMALLEST POSSIBLE atomic sub-problems (each addressing EXACTLY ONE thing)
        3. Test each sub-problem by asking: "Is this addressing exactly one specific requirement?"
        4. If a sub-problem addresses multiple things, split it further
        5. Connect each piece of feedback to specific atomic sub-problems using direct quotes
        6. Match relevant filter values to each atomic sub-problem
        
        Show your detailed reasoning process, highlighting how you:
        1. Interpreted the feedback
        2. Broke the query into truly atomic sub-problems
        3. Connected feedback to each atomic element
        4. Considered existing filter values in your analysis
        """

        messages = [
            {"role": "system", "content": self.system_prompt},
            {"role": "user", "content": prompt}
        ]
        
        response = self.call_llm(messages)
        response = eval(response[response.find('{'):response.rfind('}') + 1])
        return response


class FilterAwareSubProblemAnalysisAgent(Agent):
    def __init__(self, api_key: str):
        super().__init__(api_key)
        self.system_prompt = """You are an expert in solving specific recruitment sub-problems, with the ability to incorporate targeted feedback and existing filter values.
        IMPORTANT: Focus ONLY on the specific sub-problem assigned to you. While you may reference
        the original query for context, your solution should be specifically targeted to your assigned sub-problem.

        ## FILTER SYSTEM REFERENCE GUIDE
        This comprehensive guide explains all available filters and their proper usage:


        1. SKILL/KEYWORDS
        - Purpose: Match profiles containing specific skills, expertise, or specializations
        - Format: List of strings (e.g., ["Python", "project management", "recruiting"])
        - Usage: Profiles must contain at least ONE of the listed skills/keywords
        - Example: {"SKILL": ["Python", "machine learning", "data science"]}

        2. INDUSTRY
        - Purpose: Filter profiles by industry experience
        - Format: List of strings (e.g., ["Finance", "Technology", "Healthcare"])
        - Usage: Displays profiles with experience in any of the specified industries
        - Example: {"INDUSTRY": ["Healthcare", "Pharmaceuticals"]}

        3. COMPANY
        - Purpose: Filter by current or past company association
        - Format: List of strings representing companies or company characteristics
        - Usage: Identifies relevant companies based on extracted information
        - Example: {"COMPANY": ["Microsoft and similar companies", "companies based in USA"]}

        4. LOCATION
        - Purpose: Filter profiles by geographic location
        - Format: List of strings (e.g., ["New York", "Europe", "Countries near Egypt"])
        - Usage: Identifies specific locations or regions, including nearby areas when specified
        - Example: {"LOCATION": ["San Francisco", "Bay Area"]}

        5. EDUCATION
        - Purpose: Filter by academic qualifications
        - Format: List of dictionaries with degree and optional major
        - Usage: Matches profiles with specific educational backgrounds
        - Example: {"EDUCATION": [{"degree": "Master's", "major": "Business Administration"}]}

        6. NAME
        - Purpose: Search for specific individuals by name
        - Format: List of strings (e.g., ["John Doe", "Will Smith"])
        - Usage: Only returns profiles with exact name matches
        - Example: {"NAME": ["John Smith", "Maria Garcia"]}

        7. SCHOOL
        - Purpose: Filter by educational institution
        - Format: List of strings (e.g., ["Stanford University", "Yale University"])
        - Usage: Profiles must be associated with at least one of the specified schools
        - Example: {"SCHOOL": ["Harvard University", "MIT"]}

        8. COMPANY_TENURE
        - Purpose: Filter by length of time in CURRENT company or industry
        - Format: String representing duration (e.g., "2 years", "5+ years")
        - Usage: Only shows profiles with specified tenure in their current company
        - Example: {"COMPANY_TENURE": "3+ years"}

        9. ROLE_TENURE
        - Purpose: Filter by length of time in CURRENT role or position
        - Format: String representing duration (e.g., "2 years", "5+ years")
        - Usage: Only shows profiles with specified tenure in their current role
        - Example: {"ROLE_TENURE": "2+ years"}

        10. TOTAL_WORKING_YEARS
        - Purpose: Filter by overall career duration
        - Format: List of two integers representing minimum and maximum years
        - Usage: Shows profiles with total experience within the specified range
        - Example: {"TOTAL_WORKING_YEARS": [5, 10]}

        11. GENDER
        - Purpose: Filter profiles by gender
        - Format: List containing "Female" if female-only search is required
        - Usage: When specified as ["Female"], only female profiles are shown
        - Example: {"GENDER": ["Female"]}

        12. AGE
        - Purpose: Filter profiles by age category
        - Format: List of strings from predefined categories: "Under 25", "Over 50", "Over 65"
        - Usage: Shows profiles matching the specified age categories
        - Example: {"AGE": ["Over 50"]}

        13. ETHNICITY
        - Purpose: Filter profiles by ethnic background
        - Format: List of strings from predefined ethnicities: "Asian", "African", "Hispanic", etc.
        - Usage: Only applied when explicitly mentioned in the context of ethnicity
        - Example: {"ETHNICITY": ["South Asian", "Hispanic"]}

        14. CURRENT_OWNERSHIP
        - Purpose: Filter by company ownership type
        - Format: List of strings from: "Public", "Private", "VC Funded", "Private Equity Backed"
        - Usage: Only applied when explicitly mentioned; shows profiles at companies with that ownership
        - Example: {"CURRENT_OWNERSHIP": ["VC Funded"]}

        15. JOB_TITLE/BUSINESS_FUNCTION
        - Purpose: Filter by job roles or business functions
        - Format: List of strings representing job titles or functions
        - Usage: Matches specific roles or entire functional areas
        - Example: {"JOB_TITLE": ["Chief Sales Officer", "Sales Director"]}

        16. MANAGEMENT_LEVEL
        - Purpose: Filter by management hierarchy
        - Format: List of strings from predefined levels like "C-Suite/Chiefs", "Director", etc.
        - Usage: Only extracted when user asks for entire management domains
        - Example: {"MANAGEMENT_LEVEL": ["C-Suite/Chiefs", "Executive VP or Sr. VP"]}


        For this specific sub-problem, follow this detailed analysis process:
        1. Initial Assessment
           - Understand the specific requirement of THIS sub-problem
           - Review any specific feedback provided for this sub-problem
           - Review any existing filter values relevant to this sub-problem
           - Identify relevant filter categories for THIS component
           - Review dependencies and constraints specific to THIS aspect

        2. Feedback and Filter Integration
           - Carefully analyze how the provided feedback applies to this specific sub-problem
           - Consider how existing filter values constrain or guide your solution
           - Adjust your approach based on the feedback and filter values
           - Address any issues or improvements mentioned in the feedback
           - Build upon or refine existing filter values

        3. Solution Generation
           - FILTER-FIRST APPROACH: Each solution MUST manipulate specific filters
           - For each solution, identify exactly which filters to use and what values to assign
           - ALL solutions must involve creating, modifying, or refining filter values
           - Be precise about filter values - use exact formats as shown in the reference guide
           - Generate multiple filter-based approaches for THIS specific requirement
           - Consider edge cases and limitations
           - Ensure solution stays focused on the assigned sub-problem
           - Apply the lessons learned from feedback
           - Consider compatibility with existing filter values
        
        4. Solution Evaluation
           - Score each filter-based solution (1-10)
           - List pros and cons specific to this filter approach
           - Consider implementation feasibility of the filter configuration
           - Evaluate how well the filter solution addresses the feedback
           - Evaluate how well the filter solution integrates with existing filter values

        5. Final Selection
           - Choose optimal filter-based solution for this specific component
           - Document reasoning for this specific filter configuration
           - Provide confidence score for the selected filter approach
           - Explain how feedback influenced your final filter choice
           - Explain how existing filter values influenced your final filter configuration

        CRITICAL: Every proposed solution MUST involve manipulating one or more filters. Solutions that don't directly specify filter actions are invalid.

        Remember: Stay focused on your assigned sub-problem. Don't try to solve the entire query.

        Output structure:
        {
            "sub_problem_analysis": {
                "original_context": "",
                "thought_process": "",
                "interpreted_requirement": "",
                "relevant_filters": {
                    "identified_filters": [],
                    "rationale": ""
                },
                "filter_integration": "",
                "feedback_integration": "",
                "proposed_solutions": [
                    {
                        "thinking": "",
                        "filter_actions": {
                            "filter_name": "description of action with specific values"
                        },
                        "feedback_addressed": "",
                        "implementation_notes": "",
                        "approach": "",
                        "score": 0,
                        "pros": [],
                        "cons": [],
                        "filter_compatibility": ""
                    }
                ],
                "selected_solution": {
                    "thinking_for_selection": "",
                    "filter_values": {
                        "filter_name": "specific_value"
                    },
                    "confidence": 0.0,
                    "reasoning": "",
                    "validation_notes": [],
                    "feedback_influence": "",
                    "filter_influence": ""
                }
            }
        }"""

    async def solve_async(self, sub_problem: SubProblem, query_context: QueryContext) -> Solution:
        feedback_section = ""
        if sub_problem.relevant_feedback:
            feedback_section = f"""
            The following feedback is specifically relevant to this sub-problem:
            {sub_problem.relevant_feedback}
            
            Make sure your solution directly addresses this feedback.
            """
        elif query_context.feedback:
            feedback_section = f"""
            General feedback on the query (consider only what's relevant to this sub-problem):
            {query_context.feedback}
            """

        filter_section = ""
        if sub_problem.relevant_filters:
            filter_section = f"""
            The following filter values are specifically relevant to this sub-problem:
            {json.dumps(sub_problem.relevant_filters, indent=2)}
            
            Consider these filter values when developing your solution.
            You can build upon, refine, or modify these filter values as needed.
            """
        elif query_context.filter_values:
            filter_section = f"""
            General filter values from previous processing (consider only what's relevant to this sub-problem):
            {json.dumps(query_context.filter_values, indent=2)}
            """

        prompt = f"""Focus on solving this specific sub-problem:

        Sub-problem Category: {sub_problem.category}
        Sub-problem Description: {sub_problem.description}
        Specific Query Segment to Address: {sub_problem.query_segment}

        {feedback_section}
        
        {filter_section}

        For reference only, original query context: {query_context.original_query}

        IMPORTANT INSTRUCTIONS:
        1. Your solution MUST involve manipulating specific filters
        2. Each proposed solution must specify exactly which filters to create/modify/refine
        3. Be precise about filter values and formats according to the reference guide
        4. Every solution must result in concrete filter values
        5. Don't just describe what to do - specify exact filter configurations

        Remember: Focus only on solving this specific sub-problem component.
        Do not attempt to solve the entire query.

        Show your detailed reasoning process for this specific component,
        including how you've incorporated the relevant feedback and filter values."""

        messages = [
            {"role": "system", "content": self.system_prompt},
            {"role": "user", "content": prompt}
        ]
        
        response = await self.call_llm_async(messages)
        response = (response[response.find('{'):response.rfind('}') + 1])
        return json.loads(response)
    
    
class IntegrationAgent(Agent):
    def __init__(self, api_key: str):
        super().__init__(api_key)
        self.system_prompt = """You are an expert in integrating and validating recruitment filter solutions and converting them into specific JSON filter formats.

        Your critical responsibilities:
        1. Analyze all solutions thoroughly
        2. Provide detailed reasoning for each decision
        3. Convert the solutions into the exact filter format
        4. Only include relevant filters with valid values
        5. Ensure all values are logically consistent
        6. Consider feedback and existing filter values in your integration

        The final output must strictly follow this filter structure:
        {
            "job_title/business_function": {
                "current": [{"title_name": "", "min_staff": 0, "max_staff": 50000000}],
                "past": [{"title_name": "", "min_staff": 0, "max_staff": 50000000}],
                "both": [{"title_name": "", "min_staff": 0, "max_staff": 50000000}],
                "event": ""  # Can only be "CURRENT", "PAST", "CURRENT OR PAST", "CURRENT AND PAST"
            },
            "management_level": {
                "current": [],
                "past": [],
                "both": [],
                "event": ""
            },
            "location": {
                "current": [],
                "past": [],
                "both": [],
                "event": ""
            },
            "company": {
                "current_prompt": "",
                "past_prompt": "",
                "event": ""
            },
            "education": [{"degree": "", "major": ""}],
            "school": [],
            "current_ownership": [],
            "name": [],
            "ethnicity": [],
            "age": [],
            "total_working_years": {"min": null, "max": null},
            "role_tenure": {"min": null, "max": null},
            "company_tenure": {"min": null, "max": null},
            "skills": []
        }

        Before providing the final JSON:
        1. Analyze each requirement thoroughly
        2. Explain your reasoning for each filter value
        3. Discuss any assumptions or implicit requirements
        4. Validate logical consistency
        5. Only include filters that have valid values
        6. Explain any complex decisions or trade-offs
        7. Discuss how feedback has been addressed
        8. Explain how existing filter values have been incorporated

        Your response should follow this structure:
        1. DETAILED ANALYSIS
        2. REASONING FOR EACH FILTER
        3. ASSUMPTIONS AND IMPLICATIONS
        4. VALIDATION CHECKS
        5. FEEDBACK INCORPORATION
        6. FINAL FILTER JSON
        """

    def integrate(self, solutions: List[Solution], query_context: QueryContext) -> Dict:
        print("#"*47)
        print(solutions)
        print("#"*47)
        
        feedback_section = ""
        if query_context.feedback:
            feedback_section = f"""
            Previous feedback on the query solution:
            {query_context.feedback}
            
            Ensure your integration addresses this feedback.
            """
            
        filter_section = ""
        if query_context.filter_values:
            filter_section = f"""
            Existing filter values from previous processing:
            {json.dumps(query_context.filter_values, indent=2)}
            
            Consider these filter values in your integration.
            You can build upon, refine, or modify these filter values as needed.
            """
            
        prompt = f"""Integrate these solutions while maintaining alignment with the original query:

        Original Query: {query_context.original_query}

        {feedback_section}
        
        {filter_section}

        Please provide:
        1. Detailed analysis of how each solution component fits into the final filter structure
        2. Explicit reasoning for each filter value being set
        3. Discussion of any assumptions or implicit requirements
        4. Explanation of any trade-offs or decisions made
        5. Explanation of how feedback has been addressed
        6. Explanation of how existing filter values have been incorporated
        7. Final filter JSON in the exact specified format

        Number of Solutions to Integrate: {len(solutions)}

        Solutions: {solutions}

        For each solution component, think through:
        - How does this map to our filter structure?
        - What are the explicit vs implicit requirements?
        - Are there any temporal aspects to consider (current vs past)?
        - What are the logical dependencies between different filters?
        - Are there any potential conflicts to resolve?
        - How does the feedback influence this component?
        - How do existing filter values influence this component?

        Show your complete reasoning process before providing the final JSON structure.
        """

        messages = [
            {"role": "system", "content": self.system_prompt},
            {"role": "user", "content": prompt}
        ]
        
        response = self.call_llm(messages)
        # Split the response into reasoning and JSON parts
        parts = response.split("FINAL FILTER JSON:")
        reasoning = parts[0].strip()
        
        # Parse the JSON string
        json_str = (response[response.find('{'):response.rfind('}') + 1])
        filter_structure = json.loads(json_str)
        
        return {
            "reasoning_and_analysis": reasoning,
            "filter_structure": filter_structure
        }

# Updated Recruitment Query Processor with filter values
class FilterAwareRecruitmentQueryProcessor:
    def __init__(self, api_key: str):
        self.intent_agent = FilterAwareIntentAnalysisAgent(api_key)
        self.subproblem_agent = FilterAwareSubProblemAnalysisAgent(api_key)
        self.integration_agent = IntegrationAgent(api_key)

    async def process_sub_problems(self, sub_problems: List[Dict], query_context: QueryContext) -> List[Solution]:
        # Create tasks for all sub-problems
        tasks = []
        for sub_problem_dict in sub_problems:
            sub_problem = SubProblem(**sub_problem_dict)
            task = self.subproblem_agent.solve_async(sub_problem, query_context)
            tasks.append(task)
        
        # Execute all tasks concurrently
        solutions = await asyncio.gather(*tasks)
        return solutions

    async def process_query_async(self, query: str, feedback: Optional[str] = None, filter_values: Optional[Dict[str, Any]] = None) -> Dict:
        query_context = QueryContext(
            original_query=query,
            timestamp=datetime.now().isoformat(),
            query_id=f"query_{hash(query)}",
            feedback=feedback,
            filter_values=filter_values
        )
        
        logger.info(f"Processing query: {query_context.query_id}")
        logger.info(f"Feedback provided: {'Yes' if feedback else 'No'}")
        logger.info(f"Filter values provided: {'Yes' if filter_values else 'No'}")

        try:
            # Step 1: Analyze intent and decompose, incorporating feedback and filter values
            print('-'*47)
            print("Analyzing Intent and Decomposing the Problem (with feedback and filter values)...")
            analysis = self.intent_agent.analyze(query_context)
            logger.info(f"Query decomposed into {len(analysis['sub_problems'])} sub-problems")

            # Step 2: Process all sub-problems concurrently, with relevant feedback and filter values
            print('-'*47)
            print("Processing the Sub-Problems (with feedback and filter values)...")
            solutions = await self.process_sub_problems(analysis['sub_problems'], query_context)
            logger.info(f"Generated solutions for all sub-problems")

            # Step 3: Integrate solutions
            print('-'*47)
            print("Integrating Solutions of Sub-Problems...")
            final_result = self.integration_agent.integrate(solutions, query_context)
            logger.info("Solutions integrated successfully")

            print('-'*47)
            print("Printing Final Result...")
            print(json.dumps(final_result, indent=2))

            return {
                "query_context": vars(query_context),
                "analysis": analysis,
                "solutions": solutions,
                "final_result": final_result
            }

        except Exception as e:
            logger.error(f"Error processing query: {e}")
            raise

# Modified main function to demonstrate feedback and filter values
async def main():
    _ = load_dotenv()
    api_key = os.getenv("OPENAI_API_KEY")
    processor = FilterAwareRecruitmentQueryProcessor(api_key)
    
    # Example of filter values from a previous run
    example_filter_values = {
        "job_title/business_function": {
            "current": [{"title_name": "Sales Executive", "min_staff": 0, "max_staff": 50000000}],
            "past": [],
            "both": [],
            "event": "CURRENT"
        },
        "management_level": {
            "current": [],
            "past": [],
            "both": [],
            "event": ""
        },
        "location": {
            "current": [],
            "past": [],
            "both": [],
            "event": ""
        }
    }
    
    queries = [
        # {
        #     "query": "Find sales executives.",
        #     "feedback": "We need to focus on senior sales executives only.",
        #     "filter_values": example_filter_values
        # },
        # {
        #     "query": "Find VP-level leaders from companies in Western Europe with over 30 years of experience and expertise in corporate finance.",
        #     "feedback": "The results included too many people from Eastern Europe. Also, we need to see people who specifically have M&A experience.",
        #     "filter_values": None
        # },
        {
            "query": "Find me a Chief Marketing Officer with a strong track record in scaling AI-driven SaaS platforms. Preferably based in the U.S.",
            "feedback": "People who are not CMOs are appearing a lot. The results included too many people from US. Also, we need to see people who specifically have worked in sales in the past.",
            "filter_values": """{
    "jobTitles": {
        "innerLogic": "OR",
        "current": {
            "isStrict": false,
            "isRemoved": [],
            "isSelected": [
                {
                    "max": 50000000,
                    "min": 1,
                    "name": "CMO"
                },
                {
                    "max": 50000000,
                    "min": 1,
                    "name": "Chief Marketing Officer"
                },
                {
                    "max": 50000000,
                    "min": 1,
                    "name": "Director of Marketing"
                },
                {
                    "max": 50000000,
                    "min": 1,
                    "name": "Marketing Executive"
                },
                {
                    "max": 50000000,
                    "min": 1,
                    "name": "Vice President of Marketing"
                },
                {
                    "max": 50000000,
                    "min": 1,
                    "name": "Head of Marketing"
                },
                {
                    "max": 50000000,
                    "min": 1,
                    "name": "Marketing Director"
                },
                {
                    "max": 50000000,
                    "min": 1,
                    "name": "Senior Vice President of Marketing"
                },
                {
                    "max": 50000000,
                    "min": 1,
                    "name": "Senior Marketing Director"
                }
            ]
        },
        "previous": {
            "isStrict": false,
            "isRemoved": [],
            "isSelected": []
        }
    },
    "company": {
        "innerLogic": "OR",
        "current": {
            "isStrict": false,
            "isRemoved": [],
            "isSelected": []
        },
        "previous": {
            "isStrict": false,
            "isRemoved": [],
            "isSelected": []
        }
    },
    "locations": {
        "innerLogic": "OR",
        "current": {
            "isStrict": false,
            "isRemoved": [],
            "isSelected": [
                {
                    "id": 103644278,
                    "urn": "urn:li:fs_geo:103644278",
                    "name": "United States"
                }
            ]
        },
        "previous": {
            "isStrict": false,
            "isRemoved": [],
            "isSelected": []
        }
    },
    "managementTitles": {
        "innerLogic": "OR",
        "current": {
            "isStrict": false,
            "isRemoved": [],
            "isSelected": [
                {
                    "name": "C Suite"
                },
                {
                    "name": "Executive and Sr. VP"
                },
                {
                    "name": "VP"
                },
                {
                    "name": "Head"
                },
                {
                    "name": "Director"
                },
                {
                    "name": "President"
                }
            ]
        },
        "previous": {
            "isStrict": false,
            "isRemoved": [],
            "isSelected": []
        }
    },
    "education": {
        "innerLogic": "OR",
        "current": {
            "isStrict": false,
            "isRemoved": [],
            "isSelected": []
        },
        "previous": {
            "isStrict": false,
            "isRemoved": [],
            "isSelected": []
        }
    },
    "companyOwnershipTitles": {
        "innerLogic": "OR",
        "current": {
            "isStrict": false,
            "isRemoved": [],
            "isSelected": []
        },
        "previous": {
            "isStrict": false,
            "isRemoved": [],
            "isSelected": []
        }
    },
    "university": {
        "innerLogic": "OR",
        "current": {
            "isStrict": false,
            "isRemoved": [],
            "isSelected": []
        },
        "previous": {
            "isStrict": false,
            "isRemoved": [],
            "isSelected": []
        }
    },
    "industries": {
        "innerLogic": "OR",
        "current": {
            "isStrict": false,
            "isRemoved": [],
            "isSelected": [
                {
                    "name": "Enterprise SaaS/Software"
                },
                {
                    "name": "Artificial Intelligence/Machine Learning Companies"
                },
                {
                    "name": "Marketing Technology"
                },
                {
                    "name": "Cloud Computing/Services"
                }
            ]
        },
        "previous": {
            "isStrict": false,
            "isRemoved": [],
            "isSelected": []
        }
    },
    "skills": {
        "innerLogic": "OR",
        "current": {
            "isStrict": false,
            "isRemoved": [],
            "isSelected": []
        },
        "previous": {
            "isStrict": false,
            "isRemoved": [],
            "isSelected": []
        },
        "either": {
            "isStrict": false,
            "isRemoved": [],
            "isSelected": [
                {
                    "name": "SaaS marketing"
                },
                {
                    "name": "AI/ML product marketing"
                },
                {
                    "name": "Marketing automation"
                },
                {
                    "name": "Growth marketing"
                },
                {
                    "name": "B2B technology marketing"
                },
                {
                    "name": "Scaling Teams"
                },
                {
                    "name": "Growth Strategy"
                }
            ]
        }
    },
    "product": {
        "innerLogic": "OR",
        "current": {
            "isStrict": false,
            "isRemoved": [],
            "isSelected": []
        },
        "previous": {
            "isStrict": false,
            "isRemoved": [],
            "isSelected": []
        }
    },
    "profileNames": {
        "isSelected": [],
        "isRemoved": []
    },
    "experience": {
        "total": {
            "max": 100,
            "min": 5
        }
    },
    "roleTenure": {
        "total": {
            "min": 1,
            "max": 100
        }
    },
    "companyTenure": {
        "total": {
            "max": 100,
            "min": 3
        }
    },
    "demographic": {
        "innerLogic": "OR",
        "current": {
            "isStrict": false,
            "isRemoved": [],
            "isSelected": []
        },
        "previous": {
            "isStrict": false,
            "isRemoved": [],
            "isSelected": []
        }
    },
    "isCompanyTenureSelected": true,
    "isRoleTenureSelected": false,
    "isExperienceSelected": true,
    "isCompanyOwnershipSelected": false
}"""
        },
    ]
    
    query_results = []
    for query_item in queries:
        print('-'*47)
        print("QUERY: ", query_item["query"])
        print("FEEDBACK: ", query_item["feedback"])
        print("FILTER VALUES: ", "Provided" if query_item["filter_values"] else "None")
        
        try:
            result = await processor.process_query_async(
                query_item["query"], 
                query_item["feedback"],
                query_item["filter_values"]
            )
            query_results.append({
                "query": query_item["query"], 
                "feedback": query_item["feedback"],
                "filter_values": query_item["filter_values"],
                "result": result
            })
        except Exception as e:
            traceback.print_exc()
            print(f"Error processing query: {e}")
    
    return query_results

# If you just want to test a single query with feedback and filter values
async def test_single_query():
    _ = load_dotenv()
    api_key = os.getenv("OPENAI_API_KEY")
    processor = FilterAwareRecruitmentQueryProcessor(api_key)
    
    query = "Find VP-level leaders from AI companies in Western Europe with over 30 years of experience and expertise in corporate finance who have worked previously as Salesman."
    feedback = "The results included too many professionals from big tech companies like Google and Microsoft. We need to focus on smaller AI startups and scale-ups. Also, 'salesman' is too junior - we need people who were previously in sales management roles, not individual contributors."
    
    # Example filter values
    filter_values = {
        "job_title/business_function": {
            "current": [{"title_name": "VP", "min_staff": 0, "max_staff": 50000000}],
            "past": [{"title_name": "Salesman", "min_staff": 0, "max_staff": 50000000}],
            "both": [],
            "event": "CURRENT AND PAST"
        },
        "location": {
            "current": [{"continent": "Europe", "region": "Western Europe"}],
            "past": [],
            "both": [],
            "event": "CURRENT"
        },
        "total_working_years": {"min": 30, "max": null}
    }
    
    result = await processor.process_query_async(query, feedback, filter_values)
    return result

if __name__ == "__main__":
    results = await (main())
    # Or for testing single query:
    # result = asyncio.run(test_single_query())
    
    with open("k_filter_feedback_results.json", "w") as f:
        json.dump(results, f, indent=2)

INFO:__main__:Processing query: query_-530054364588539445
INFO:__main__:Feedback provided: Yes
INFO:__main__:Filter values provided: Yes


-----------------------------------------------
QUERY:  Find me a Chief Marketing Officer with a strong track record in scaling AI-driven SaaS platforms. Preferably based in the U.S.
FEEDBACK:  People who are not CMOs are appearing a lot. The results included too many people from US. Also, we need to see people who specifically have worked in sales in the past.
FILTER VALUES:  Provided
-----------------------------------------------
Analyzing Intent and Decomposing the Problem (with feedback and filter values)...
[
  {
    "role": "system",
    "content": "You are an expert recruitment query analyzer specializing in breaking down complex queries into ATOMIC logical sub-problems while prioritizing feedback incorporation and considering existing filter values.\n\n        Your PRIMARY RESPONSIBILITIES (in order of importance):\n        1. FEEDBACK FOCUS: Deeply analyze any provided feedback and make it the central guiding principle of your analysis\n        2. ATOMICITY: Break down querie

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:__main__:Query decomposed into 4 sub-problems


```json
{
    "main_intent": "To find a Chief Marketing Officer with a proven track record in scaling AI-driven SaaS platforms, emphasizing sales experience, and based in the U.S.",
    "feedback_analysis": "The feedback highlights three critical areas: ensuring candidates are CMOs, reducing candidates from the U.S., and adding a requirement for past sales experience. This feedback dictates a shift in focus towards verifying job titles strictly, considering a more diverse location base, and emphasizing sales experience.",
    "filter_analysis": "The existing filters already include various job titles beyond CMO, which may contribute to irrelevant results. The location filter is strictly set to the U.S., which must be adjusted as per feedback. Current industry and skills filters are aligned with SaaS and AI-driven platforms but lack specificity for sales experience.",
    "thinking": "Initially, I focused on the feedback to understand the need for a strict CMO title, diversified locatio

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:__main__:Generated solutions for all sub-problems


-----------------------------------------------
Integrating Solutions of Sub-Problems...
###############################################
[{'sub_problem_analysis': {'original_context': 'Find me a Chief Marketing Officer with a strong track record in scaling AI-driven SaaS platforms. Preferably based in the U.S.', 'thought_process': 'The requirement is to filter candidates specifically holding the title of Chief Marketing Officer (CMO). The feedback indicates that profiles without this title are being included, so the solution must ensure that only profiles with the exact title of CMO are returned.', 'interpreted_requirement': "Ensure that only candidates with the job title 'Chief Marketing Officer' are included in the results.", 'relevant_filters': {'identified_filters': ['JOB_TITLE'], 'rationale': "The job title filter is directly relevant, as it allows us to specify the exact title required, which is 'Chief Marketing Officer'."}, 'filter_integration': "The existing filter value is 'CM

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:__main__:Solutions integrated successfully


1. **DETAILED ANALYSIS**

The integration of these solutions involves understanding and interpreting the requirements, feedback, and existing filter values to ensure the final filters are comprehensive and precise. The main components to consider are job title specificity, broadening location scope, incorporating past sales experience, and ensuring candidates have scaling experience in AI-driven SaaS platforms.

- **Job Title Specificity**: The feedback emphasizes the need to strictly filter for "Chief Marketing Officer" (CMO) to avoid non-CMO profiles. This requires refining the job title filters to include both the full title and its common abbreviation.

- **Location Scope**: The feedback suggests broadening the candidate pool beyond the U.S. while still considering U.S.-based candidates. This requires expanding the location filters to include international regions alongside the U.S.

- **Sales Experience**: The requirement is for candidates to have past sales experience, necessitat

In [ ]:
# from typing import Dict, List, Any, Optional
# from dataclasses import dataclass
# import openai
# import json
# from enum import Enum
# from datetime import datetime
# import logging
# from dotenv import load_dotenv
# import os
# from openai import OpenAI
# import ast
# from pprint import pprint
# from openai import AsyncOpenAI
# import asyncio
# import traceback

# logging.basicConfig(level=logging.INFO)
# logger = logging.getLogger(__name__)

# @dataclass
# class QueryContext:
#     original_query: str
#     timestamp: str
#     query_id: str
#     feedback: Optional[str] = None
#     learnings: Optional[str] = None
#     filter_values: Optional[Dict[str, Any]] = None  # Added filter_values field
    
# @dataclass
# class SubProblem:
#     category: str
#     description: str
#     dependencies: List[str]
#     priority: int
#     context_reference: Dict[str, Any]
#     query_segment: str
#     relevant_feedback: Optional[str] = None
#     relevant_learnings: Optional[str] = None
#     relevant_filters: Optional[Dict[str, Any]] = None  # Added relevant_filters field

# @dataclass
# class Solution:
#     sub_problem: SubProblem
#     proposed_solution: Dict
#     confidence: float
#     reasoning: str
#     validation_notes: List[str]

# class Agent:
#     def __init__(self, api_key: str):
#         self.client = OpenAI(api_key=api_key)
#         self.async_client = AsyncOpenAI(api_key=api_key)
        
#     def call_llm(self, messages: List[Dict]) -> str:
#         try:
#             print(json.dumps(messages, indent=2))
#             response = self.client.chat.completions.create(
#                 model="gpt-4o",
#                 messages=messages,
#                 temperature=0.7
#             )
#             print((response.choices[0].message.content))
#             return response.choices[0].message.content
#         except Exception as e:
#             logger.error(f"Error in LLM call: {e}")
#             raise

#     async def call_llm_async(self, messages: List[Dict]) -> str:
#         try:
#             response = await self.async_client.chat.completions.create(
#                 model="gpt-4o",
#                 messages=messages,
#                 temperature=0.7
#             )
#             return response.choices[0].message.content
#         except Exception as e:
#             logger.error(f"Error in async LLM call: {e}")
#             raise


# class FilterAwareIntentAnalysisAgent(Agent):
#     def __init__(self, api_key: str):
#         super().__init__(api_key)
#         self.system_prompt = """You are an expert recruitment query analyzer specializing in breaking down complex queries into ATOMIC logical sub-problems while prioritizing feedback incorporation and considering existing filter values.

#         Your PRIMARY RESPONSIBILITIES (in order of importance):
#         1. FEEDBACK FOCUS: Deeply analyze any provided feedback and make it the central guiding principle of your analysis
#         2. ATOMICITY: Break down queries into truly atomic sub-problems (each addressing ONE specific aspect only)
#         3. FILTER AWARENESS: Carefully consider existing filter values when analyzing the query
#         4. LEARNING INTEGRATION: Apply relevant system learnings to each atomic sub-problem
#         5. INTENT UNDERSTANDING: Comprehend the main intent of the recruitment query
        
#         Key principles for FEEDBACK incorporation:
#         - Treat feedback as the highest priority guidance for your analysis
#         - Explicitly connect each piece of feedback to specific atomic sub-problems
#         - If feedback contradicts the original query, prioritize the feedback's direction
#         - Use feedback to refine, modify, or completely change your understanding of requirements
#         - For each atomic sub-problem, include SPECIFIC quotes from the feedback that are relevant
        
#         Key principles for ATOMIC SUB-PROBLEM creation:
#         - Each sub-problem must address EXACTLY ONE specific requirement or concept
#         - If a potential sub-problem addresses multiple requirements, split it further
#         - Test each sub-problem with the question: "Does this solve exactly one thing?"
#         - Avoid compound requirements within a single sub-problem
#         - Ensure each sub-problem has a singular, clear focus
        
#         Key principles for LEARNING INTEGRATION:
#         - For each sub-problem, identify SPECIFIC relevant learnings that apply
#         - Only associate learnings that directly inform or constrain the specific sub-problem
#         - Extract only the portions of learnings that are relevant to each atomic element
#         - Prioritize learnings that address similar requirements to the current sub-problem
#         - Connect each learning to the specific reasoning it enables for the sub-problem
        
#         For each ATOMIC sub-problem, you must:
#         1. Focus on a single, specific aspect of the requirement
#         2. Connect relevant feedback specifically to this atomic element
#         3. Identify which specific filter values (if any) relate to this single element
#         4. Extract relevant system learnings specifically applicable to this element
#         5. Assign appropriate priority (1-5, 1 being highest)
#         6. Explicitly state dependencies on other atomic sub-problems
        
#         Your analysis process should follow these steps:
#         1. First, thoroughly analyze the feedback to identify key guidance points
#         2. Second, examine existing filter values to understand constraints
#         3. Third, review system learnings to identify relevant past solutions
#         4. Fourth, break down the query into the smallest possible atomic sub-problems
#         5. Finally, connect feedback points, filter values, and system learnings to each atomic sub-problem
        
#         Output JSON structure:
#         {
#             "main_intent": "Clear statement of primary goal",
#             "feedback_analysis": "COMPREHENSIVE analysis of how feedback modifies the query - this should be DETAILED",
#             "filter_analysis": "Analysis of how existing filter values constrain the solution",
#             "learnings_analysis": "Analysis of how system learnings inform the solution approach",
#             "thinking": "Step-by-step breakdown of how you arrived at your atomic sub-problems",
#             "analysis_notes": ["Detailed thoughts and reasoning"],
#             "potential_challenges": ["Identified challenges"],
#             "sub_problems": [
#                 {
#                     "category": "job_title/location/education/etc",
#                     "description": "Description of this SINGLE atomic requirement",
#                     "dependencies": ["other_category_ids"],
#                     "priority": "1-5",
#                     "context_reference": {"query_segment": "relevant part of original query"},
#                     "query_segment": "exact text from original query",
#                     "relevant_feedback": "SPECIFIC feedback directly relevant to this atomic sub-problem",
#                     "relevant_filters": {"filter_name": "filter_value"},
#                     "relevant_learnings": ["SPECIFIC learnings directly applicable to this atomic sub-problem"]
#                 }
#             ],
#             "confidence_score": 0.0
#         }"""

#     def analyze(self, query_context: QueryContext) -> Dict:
#         feedback_section = ""
#         if query_context.feedback:
#             feedback_section = f"""
#             CRITICAL FEEDBACK INFORMATION:
#             The following feedback has been provided about previous attempts to solve this query:
#             {query_context.feedback}
            
#             THIS FEEDBACK IS YOUR PRIMARY GUIDANCE. You must:
#             1. Make this feedback the central focus of your analysis
#             2. For each atomic sub-problem, identify specific parts of the feedback that apply to it
#             3. Use feedback to reinterpret or modify the original query when needed
#             4. If feedback conflicts with the original query, prioritize the feedback's direction
#             5. Extract specific quotes from the feedback for each atomic sub-problem
#             """
        
#         filter_section = ""
#         if query_context.filter_values:
#             filter_section = f"""
#             EXISTING FILTER VALUES:
#             {json.dumps(query_context.filter_values, indent=2)}
            
#             Consider these filter values when creating atomic sub-problems:
#             1. For each atomic sub-problem, identify specific relevant filter values
#             2. Understand how each filter constrains or guides that atomic requirement
#             3. Note where existing filters might already address parts of the query
#             """
        
#         learnings_section = ""
#         if hasattr(query_context, 'learnings') and query_context.learnings:
#             learnings_section = f"""
#             SYSTEM LEARNINGS:
#             {query_context.learnings}
            
#             Apply these learnings to your analysis:
#             1. For each atomic sub-problem, identify SPECIFIC relevant learnings
#             2. Extract only the portions of learnings that apply to each sub-problem
#             3. Use learnings to inform how you break down and categorize requirements
#             4. Connect specific learnings to specific sub-problems
#             5. Include direct references to relevant learnings for each sub-problem
#             """
        
#         prompt = f"""Analyze this recruitment query, focusing on breaking it down into TRULY ATOMIC sub-problems: 
#         {query_context.original_query}

#         {feedback_section}
        
#         {filter_section}
        
#         {learnings_section}

#         IMPORTANT ANALYSIS INSTRUCTIONS:
#         1. PRIMARILY focus on the feedback (if provided) to guide your understanding
#         2. Break the query into the SMALLEST POSSIBLE atomic sub-problems (each addressing EXACTLY ONE thing)
#         3. Test each sub-problem by asking: "Is this addressing exactly one specific requirement?"
#         4. If a sub-problem addresses multiple things, split it further
#         5. Connect each piece of feedback to specific atomic sub-problems using direct quotes
#         6. Match relevant filter values to each atomic sub-problem
#         7. Connect specific system learnings to each relevant atomic sub-problem
        
#         Show your detailed reasoning process, highlighting how you:
#         1. Interpreted the feedback
#         2. Broke the query into truly atomic sub-problems
#         3. Connected feedback to each atomic element
#         4. Considered existing filter values in your analysis
#         5. Applied system learnings to inform your sub-problem identification
#         """

#         messages = [
#             {"role": "system", "content": self.system_prompt},
#             {"role": "user", "content": prompt}
#         ]
        
#         response = self.call_llm(messages)
#         response = eval(response[response.find('{'):response.rfind('}') + 1])
#         return response
    


# class FilterAwareSubProblemAnalysisAgent(Agent):
#     def __init__(self, api_key: str):
#         super().__init__(api_key)
#         self.system_prompt = """You are an expert in solving specific recruitment sub-problems, with the ability to incorporate targeted feedback, existing filter values, and system learnings.
#         IMPORTANT: Focus ONLY on the specific sub-problem assigned to you. While you may reference
#         the original query for context, your solution should be specifically targeted to your assigned sub-problem.

#         ## FILTER SYSTEM REFERENCE GUIDE
#         This comprehensive guide explains all available filters and their proper usage:


#         1. SKILL/KEYWORDS
#         - Purpose: Match profiles containing specific skills, expertise, or specializations
#         - Format: List of strings (e.g., ["Python", "project management", "recruiting"])
#         - Usage: Profiles must contain at least ONE of the listed skills/keywords
#         - Example: {"SKILL": ["Python", "machine learning", "data science"]}

#         2. INDUSTRY
#         - Purpose: Filter profiles by industry experience
#         - Format: List of strings (e.g., ["Finance", "Technology", "Healthcare"])
#         - Usage: Displays profiles with experience in any of the specified industries
#         - Example: {"INDUSTRY": ["Healthcare", "Pharmaceuticals"]}

#         3. COMPANY
#         - Purpose: Filter by current or past company association
#         - Format: List of strings representing companies or company characteristics
#         - Usage: Identifies relevant companies based on extracted information
#         - Example: {"COMPANY": ["Microsoft and similar companies", "companies based in USA"]}

#         4. LOCATION
#         - Purpose: Filter profiles by geographic location
#         - Format: List of strings (e.g., ["New York", "Europe", "Countries near Egypt"])
#         - Usage: Identifies specific locations or regions, including nearby areas when specified
#         - Example: {"LOCATION": ["San Francisco", "Bay Area"]}

#         5. EDUCATION
#         - Purpose: Filter by academic qualifications
#         - Format: List of dictionaries with degree and optional major
#         - Usage: Matches profiles with specific educational backgrounds
#         - Example: {"EDUCATION": [{"degree": "Master's", "major": "Business Administration"}]}

#         6. NAME
#         - Purpose: Search for specific individuals by name
#         - Format: List of strings (e.g., ["John Doe", "Will Smith"])
#         - Usage: Only returns profiles with exact name matches
#         - Example: {"NAME": ["John Smith", "Maria Garcia"]}

#         7. SCHOOL
#         - Purpose: Filter by educational institution
#         - Format: List of strings (e.g., ["Stanford University", "Yale University"])
#         - Usage: Profiles must be associated with at least one of the specified schools
#         - Example: {"SCHOOL": ["Harvard University", "MIT"]}

#         8. COMPANY_TENURE
#         - Purpose: Filter by length of time in CURRENT company or industry
#         - Format: String representing duration (e.g., "2 years", "5+ years")
#         - Usage: Only shows profiles with specified tenure in their current company
#         - Example: {"COMPANY_TENURE": "3+ years"}

#         9. ROLE_TENURE
#         - Purpose: Filter by length of time in CURRENT role or position
#         - Format: String representing duration (e.g., "2 years", "5+ years")
#         - Usage: Only shows profiles with specified tenure in their current role
#         - Example: {"ROLE_TENURE": "2+ years"}

#         10. TOTAL_WORKING_YEARS
#         - Purpose: Filter by overall career duration
#         - Format: List of two integers representing minimum and maximum years
#         - Usage: Shows profiles with total experience within the specified range
#         - Example: {"TOTAL_WORKING_YEARS": [5, 10]}

#         11. GENDER
#         - Purpose: Filter profiles by gender
#         - Format: List containing "Female" if female-only search is required
#         - Usage: When specified as ["Female"], only female profiles are shown
#         - Example: {"GENDER": ["Female"]}

#         12. AGE
#         - Purpose: Filter profiles by age category
#         - Format: List of strings from predefined categories: "Under 25", "Over 50", "Over 65"
#         - Usage: Shows profiles matching the specified age categories
#         - Example: {"AGE": ["Over 50"]}

#         13. ETHNICITY
#         - Purpose: Filter profiles by ethnic background
#         - Format: List of strings from predefined ethnicities: "Asian", "African", "Hispanic", etc.
#         - Usage: Only applied when explicitly mentioned in the context of ethnicity
#         - Example: {"ETHNICITY": ["South Asian", "Hispanic"]}

#         14. CURRENT_OWNERSHIP
#         - Purpose: Filter by company ownership type
#         - Format: List of strings from: "Public", "Private", "VC Funded", "Private Equity Backed"
#         - Usage: Only applied when explicitly mentioned; shows profiles at companies with that ownership
#         - Example: {"CURRENT_OWNERSHIP": ["VC Funded"]}

#         15. JOB_TITLE/BUSINESS_FUNCTION
#         - Purpose: Filter by job roles or business functions
#         - Format: List of strings representing job titles or functions
#         - Usage: Matches specific roles or entire functional areas
#         - Example: {"JOB_TITLE": ["Chief Sales Officer", "Sales Director"]}

#         16. MANAGEMENT_LEVEL
#         - Purpose: Filter by management hierarchy
#         - Format: List of strings from predefined levels like "C-Suite/Chiefs", "Director", etc.
#         - Usage: Only extracted when user asks for entire management domains
#         - Example: {"MANAGEMENT_LEVEL": ["C-Suite/Chiefs", "Executive VP or Sr. VP"]}


#         For this specific sub-problem, follow this detailed analysis process:
#         1. Initial Assessment
#            - Understand the specific requirement of THIS sub-problem
#            - Review any specific feedback provided for this sub-problem
#            - Review any existing filter values relevant to this sub-problem
#            - Review relevant system learnings applicable to this sub-problem
#            - Identify relevant filter categories for THIS component
#            - Review dependencies and constraints specific to THIS aspect

#         2. Feedback, Filter, and Learning Integration
#            - Carefully analyze how the provided feedback applies to this specific sub-problem
#            - Consider how existing filter values constrain or guide your solution
#            - Apply specific insights from system learnings to this sub-problem
#            - Adjust your approach based on the feedback, filter values, and learnings
#            - Address any issues or improvements mentioned in the feedback
#            - Build upon or refine existing filter values
#            - Draw inspiration from system learnings when proposing solutions

#         3. Solution Generation
#            - FILTER-FIRST APPROACH: Each solution MUST manipulate specific filters
#            - Try to generate 3 proposed solution for THIS specific sub-problem
#            - For each solution, identify exactly which filters to use and what values to assign
#            - ALL solutions must involve creating, modifying, or refining filter values
#            - Be precise about filter values - use exact formats as shown in the reference guide
#            - Generate multiple filter-based approaches for THIS specific requirement
#            - Take direct inspiration from system learnings when formulating solutions
#            - Consider edge cases and limitations
#            - Ensure solution stays focused on the assigned sub-problem
#            - Apply the lessons learned from feedback
#            - Consider compatibility with existing filter values
        
#         4. Solution Evaluation
#            - Score each filter-based solution (1-10)
#            - List pros and cons specific to this filter approach
#            - Consider implementation feasibility of the filter configuration
#            - Evaluate how well the filter solution addresses the feedback
#            - Evaluate how well the filter solution integrates with existing filter values
#            - Assess how well the solution applies insights from system learnings

#         5. Final Selection
#            - Choose optimal filter-based solution for this specific component
#            - Document reasoning for this specific filter configuration
#            - Provide confidence score for the selected filter approach
#            - Explain how feedback influenced your final filter choice
#            - Explain how existing filter values influenced your final filter configuration
#            - Explain how system learnings influenced your solution selection

#         CRITICAL: Every proposed solution MUST involve manipulating one or more filters. Solutions that don't directly specify filter actions are invalid.

#         Remember: Stay focused on your assigned sub-problem. Don't try to solve the entire query.

#         Output structure:
#         {
#             "sub_problem_analysis": {
#                 "original_context": "",
#                 "thought_process": "",
#                 "interpreted_requirement": "",
#                 "relevant_filters": {
#                     "identified_filters": [],
#                     "rationale": ""
#                 },
#                 "relevant_learnings": {
#                     "applicable_learnings": [],
#                     "learning_insights": ""
#                 },
#                 "filter_integration": "",
#                 "feedback_integration": "",
#                 "learning_integration": "",
#                 "proposed_solutions": [
#                     {
#                         "thinking": "",
#                         "filter_actions": {
#                             "filter_name": "description of action with specific values"
#                         },
#                         "feedback_addressed": "",
#                         "learning_applied": "",
#                         "implementation_notes": "",
#                         "approach": "",
#                         "score": 0,
#                         "pros": [],
#                         "cons": [],
#                         "filter_compatibility": ""
#                     }
#                 ],
#                 "selected_solution": {
#                     "thinking_for_selection": "",
#                     "filter_values": {
#                         "filter_name": "specific_value"
#                     },
#                     "confidence": 0.0,
#                     "reasoning": "",
#                     "validation_notes": [],
#                     "feedback_influence": "",
#                     "filter_influence": "",
#                     "learning_influence": ""
#                 }
#             }
#         }"""
        
#         self.reflection_prompt = """You are a critical evaluator of recruitment filter solutions. Your task is to evaluate a proposed solution for a specific sub-problem and identify any issues, improvements, or inconsistencies.

#         Review the following solution critically:

#         1. Evaluate alignment with the sub-problem:
#            - Does the solution directly address the specific sub-problem?
#            - Is the solution overly broad or overly narrow?
#            - Does it stay focused on the assigned task without scope creep?

#         2. Evaluate filter usage:
#            - Are the selected filters appropriate for the requirement?
#            - Are the filter values formatted correctly according to the reference guide?
#            - Are there better filter alternatives that weren't considered?
#            - Are there any missing filters that should be included?

#         3. Evaluate feedback integration:
#            - Has the solution properly addressed the feedback?
#            - Are there aspects of the feedback being overlooked?
#            - Is the solution consistent with the direction provided in the feedback?

#         4. Evaluate existing filter integration:
#            - Does the solution properly build upon existing filter values?
#            - Are there any conflicts with existing filters?
#            - Is the solution complementary to the existing filter configuration?

#         5. Evaluate learning application:
#            - Have relevant system learnings been properly applied?
#            - Are there important learnings being overlooked?
#            - Does the solution reflect best practices from the system learnings?

#         6. Evaluate solution quality:
#            - Is the solution implementation practical and feasible?
#            - Are the pros and cons assessment accurate and comprehensive?
#            - Is the confidence score justified?
#            - Is the reasoning sound and logical?

#         7. Identify improvements:
#            - Specific ways the solution could be improved
#            - Alternative approaches that might be more effective
#            - Adjustments to make the solution more precise or comprehensive

#         Be honest, thorough, and constructive in your evaluation. Focus on identifying concrete improvements.

#         Output structure:
#         {
#             "alignment_score": 0-10,
#             "alignment_issues": [],
#             "filter_score": 0-10,
#             "filter_issues": [],
#             "feedback_score": 0-10,
#             "feedback_issues": [],
#             "existing_filter_score": 0-10,
#             "existing_filter_issues": [],
#             "learning_score": 0-10,
#             "learning_issues": [],
#             "solution_quality_score": 0-10,
#             "solution_quality_issues": [],
#             "overall_assessment": "",
#             "recommended_improvements": [],
#             "overall_score": 0-10
#         }"""

#     async def solve_async(self, sub_problem: SubProblem, query_context: QueryContext) -> Solution:
#         feedback_section = ""
#         if sub_problem.relevant_feedback:
#             feedback_section = f"""
#             The following feedback is specifically relevant to this sub-problem:
#             {sub_problem.relevant_feedback}
            
#             Make sure your solution directly addresses this feedback.
#             """
#         elif query_context.feedback:
#             feedback_section = f"""
#             General feedback on the query (consider only what's relevant to this sub-problem):
#             {query_context.feedback}
#             """

#         filter_section = ""
#         if sub_problem.relevant_filters:
#             filter_section = f"""
#             The following filter values are specifically relevant to this sub-problem:
#             {json.dumps(sub_problem.relevant_filters, indent=2)}
            
#             Consider these filter values when developing your solution.
#             You can build upon, refine, or modify these filter values as needed.
#             """
#         elif query_context.filter_values:
#             filter_section = f"""
#             General filter values from previous processing (consider only what's relevant to this sub-problem):
#             {json.dumps(query_context.filter_values, indent=2)}
#             """
        
#         learnings_section = ""
#         if hasattr(sub_problem, 'relevant_learnings') and sub_problem.relevant_learnings:
#             learnings_section = f"""
#             The following system learnings are specifically relevant to this sub-problem:
#             {sub_problem.relevant_learnings}
            
#             Use these specific learnings to inform your solution approach.
#             Draw direct inspiration from these learnings when proposing and selecting filter actions.
#             """
#         elif hasattr(query_context, 'learnings') and query_context.learnings:
#             learnings_section = f"""
#             General system learnings (consider only what's relevant to this sub-problem):
#             {query_context.learnings}
            
#             Extract insights relevant to this specific sub-problem to inform your solutions.
#             """

#         prompt = f"""Focus on solving this specific sub-problem:

#         Sub-problem Category: {sub_problem.category}
#         Sub-problem Description: {sub_problem.description}
#         Specific Query Segment to Address: {sub_problem.query_segment}

#         {feedback_section}
        
#         {filter_section}
        
#         {learnings_section}

#         For reference only, original query context: {query_context.original_query}

#         IMPORTANT INSTRUCTIONS:
#         1. Your solution MUST involve manipulating specific filters
#         2. Each proposed solution must specify exactly which filters to create/modify/refine
#         3. Be precise about filter values and formats according to the reference guide
#         4. Every solution must result in concrete filter values
#         5. Don't just describe what to do - specify exact filter configurations
#         6. Draw direct inspiration from relevant system learnings when designing solutions

#         Remember: Focus only on solving this specific sub-problem component.
#         Do not attempt to solve the entire query.

#         Show your detailed reasoning process for this specific component,
#         including how you've incorporated the relevant feedback, filter values, and system learnings."""

#         messages = [
#             {"role": "system", "content": self.system_prompt},
#             {"role": "user", "content": prompt}
#         ]
        
#         # Initial solution generation
#         response = await self.call_llm_async(messages)
#         raw_response = response
#         response_json = json.loads(response[response.find('{'):response.rfind('}') + 1])
        
#         # Self-reflection step
#         reflection_prompt = f"""Review and evaluate this solution for the following sub-problem:

#         Sub-problem Category: {sub_problem.category}
#         Sub-problem Description: {sub_problem.description}
#         Specific Query Segment to Address: {sub_problem.query_segment}

#         {feedback_section}
        
#         {filter_section}
        
#         {learnings_section}

#         Proposed Solution:
#         {json.dumps(response_json, indent=2)}

#         Critically evaluate the solution based on:
#         1. How well it addresses the specific sub-problem
#         2. Appropriate use of filters
#         3. Integration of feedback
#         4. Integration with existing filter values
#         5. Application of system learnings
#         6. Overall solution quality and practicality

#         Identify any issues, inconsistencies, or potential improvements."""

#         reflection_messages = [
#             {"role": "system", "content": self.reflection_prompt},
#             {"role": "user", "content": reflection_prompt}
#         ]
        
#         # Get reflection analysis
#         reflection_response = await self.call_llm_async(reflection_messages)
#         reflection_json = json.loads(reflection_response[reflection_response.find('{'):reflection_response.rfind('}') + 1])
        
#         # If there are significant issues, refine the solution
#         if reflection_json["overall_score"] < 7:
#             refinement_prompt = f"""You previously provided a solution for this sub-problem:

#             Sub-problem Category: {sub_problem.category}
#             Sub-problem Description: {sub_problem.description}
#             Specific Query Segment to Address: {sub_problem.query_segment}

#             Your solution had these issues:
#             {json.dumps(reflection_json["recommended_improvements"], indent=2)}

#             Overall assessment:
#             {reflection_json["overall_assessment"]}

#             Please provide an improved solution that addresses these issues.

#             {feedback_section}
            
#             {filter_section}
            
#             {learnings_section}

#             Original solution for reference:
#             {json.dumps(response_json, indent=2)}

#             Focus specifically on addressing the identified issues and improvements."""

#             refinement_messages = [
#                 {"role": "system", "content": self.system_prompt},
#                 {"role": "user", "content": refinement_prompt}
#             ]
            
#             # Get refined solution
#             refined_response = await self.call_llm_async(refinement_messages)
#             refined_json = json.loads(refined_response[refined_response.find('{'):refined_response.rfind('}') + 1])
            
#             # Add reflection and refinement information to the solution
#             refined_json["sub_problem_analysis"]["self_reflection"] = {
#                 "initial_assessment": reflection_json,
#                 "refinement_process": "Solution was refined based on self-reflection feedback",
#                 "improvements_made": reflection_json["recommended_improvements"]
#             }
            
#             return refined_json
#         else:
#             # If solution was good, just add the reflection information
#             response_json["sub_problem_analysis"]["self_reflection"] = {
#                 "assessment": reflection_json,
#                 "conclusion": "Solution was determined to be high quality, no refinement needed"
#             }
            
#             return response_json
    
    
# class IntegrationAgent(Agent):
#     def __init__(self, api_key: str):
#         super().__init__(api_key)
#         self.system_prompt = """You are an expert in integrating and validating recruitment filter solutions and converting them into specific JSON filter formats.

#         Your critical responsibilities:
#         1. Analyze all solutions thoroughly
#         2. Provide detailed reasoning for each decision
#         3. Convert the solutions into the exact filter format
#         4. Only include relevant filters with valid values
#         5. Ensure all values are logically consistent
#         6. Consider feedback and existing filter values in your integration

#         The final output must strictly follow this filter structure:
#         {
#             "job_title/business_function": {
#                 "current": [{"title_name": "", "min_staff": 0, "max_staff": 50000000}],
#                 "past": [{"title_name": "", "min_staff": 0, "max_staff": 50000000}],
#                 "both": [{"title_name": "", "min_staff": 0, "max_staff": 50000000}],
#                 "event": ""  # Can only be "CURRENT", "PAST", "CURRENT OR PAST", "CURRENT AND PAST"
#             },
#             "management_level": {
#                 "current": [],
#                 "past": [],
#                 "both": [],
#                 "event": ""
#             },
#             "location": {
#                 "current": [],
#                 "past": [],
#                 "both": [],
#                 "event": ""
#             },
#             "company": {
#                 "current_prompt": "",
#                 "past_prompt": "",
#                 "event": ""
#             },
#             "education": [{"degree": "", "major": ""}],
#             "school": [],
#             "current_ownership": [],
#             "name": [],
#             "ethnicity": [],
#             "age": [],
#             "total_working_years": {"min": null, "max": null},
#             "role_tenure": {"min": null, "max": null},
#             "company_tenure": {"min": null, "max": null},
#             "skills": []
#         }

#         Before providing the final JSON:
#         1. Analyze each requirement thoroughly
#         2. Explain your reasoning for each filter value
#         3. Discuss any assumptions or implicit requirements
#         4. Validate logical consistency
#         5. Only include filters that have valid values
#         6. Explain any complex decisions or trade-offs
#         7. Discuss how feedback has been addressed
#         8. Explain how existing filter values have been incorporated

#         Your response should follow this structure:
#         1. DETAILED ANALYSIS
#         2. REASONING FOR EACH FILTER
#         3. ASSUMPTIONS AND IMPLICATIONS
#         4. VALIDATION CHECKS
#         5. FEEDBACK INCORPORATION
#         6. FINAL FILTER JSON
#         """

#     def integrate(self, solutions: List[Solution], query_context: QueryContext) -> Dict:
#         print("#"*47)
#         print(solutions)
#         print("#"*47)
        
#         feedback_section = ""
#         if query_context.feedback:
#             feedback_section = f"""
#             Previous feedback on the query solution:
#             {query_context.feedback}
            
#             Ensure your integration addresses this feedback.
#             """
            
#         filter_section = ""
#         if query_context.filter_values:
#             filter_section = f"""
#             Existing filter values from previous processing:
#             {json.dumps(query_context.filter_values, indent=2)}
            
#             Consider these filter values in your integration.
#             You can build upon, refine, or modify these filter values as needed.
#             """
            
#         prompt = f"""Integrate these solutions while maintaining alignment with the original query:

#         Original Query: {query_context.original_query}

#         {feedback_section}
        
#         {filter_section}

#         Please provide:
#         1. Detailed analysis of how each solution component fits into the final filter structure
#         2. Explicit reasoning for each filter value being set
#         3. Discussion of any assumptions or implicit requirements
#         4. Explanation of any trade-offs or decisions made
#         5. Explanation of how feedback has been addressed
#         6. Explanation of how existing filter values have been incorporated
#         7. Final filter JSON in the exact specified format

#         Number of Solutions to Integrate: {len(solutions)}

#         Solutions: {solutions}

#         For each solution component, think through:
#         - How does this map to our filter structure?
#         - What are the explicit vs implicit requirements?
#         - Are there any temporal aspects to consider (current vs past)?
#         - What are the logical dependencies between different filters?
#         - Are there any potential conflicts to resolve?
#         - How does the feedback influence this component?
#         - How do existing filter values influence this component?

#         Show your complete reasoning process before providing the final JSON structure.
#         """

#         messages = [
#             {"role": "system", "content": self.system_prompt},
#             {"role": "user", "content": prompt}
#         ]
        
#         response = self.call_llm(messages)
#         # Split the response into reasoning and JSON parts
#         parts = response.split("FINAL FILTER JSON:")
#         reasoning = parts[0].strip()
        
#         # Parse the JSON string
#         json_str = (response[response.find('{'):response.rfind('}') + 1])
#         filter_structure = json.loads(json_str)
        
#         return {
#             "reasoning_and_analysis": reasoning,
#             "filter_structure": filter_structure
#         }

# # Updated Recruitment Query Processor with filter values and learnings
# class FilterAwareRecruitmentQueryProcessor:
#     def __init__(self, api_key: str):
#         self.intent_agent = FilterAwareIntentAnalysisAgent(api_key)
#         self.subproblem_agent = FilterAwareSubProblemAnalysisAgent(api_key)
#         self.integration_agent = IntegrationAgent(api_key)

#     async def process_sub_problems(self, sub_problems: List[Dict], query_context: QueryContext) -> List[Solution]:
#         # Create tasks for all sub-problems
#         tasks = []
#         for sub_problem_dict in sub_problems:
#             sub_problem = SubProblem(**sub_problem_dict)
#             task = self.subproblem_agent.solve_async(sub_problem, query_context)
#             tasks.append(task)
        
#         # Execute all tasks concurrently
#         solutions = await asyncio.gather(*tasks)
#         return solutions

#     async def process_query_async(self, query: str, feedback: Optional[str] = None, 
#                                   filter_values: Optional[Dict[str, Any]] = None,
#                                   learnings: Optional[str] = None) -> Dict:
#         query_context = QueryContext(
#             original_query=query,
#             timestamp=datetime.now().isoformat(),
#             query_id=f"query_{hash(query)}",
#             feedback=feedback,
#             filter_values=filter_values,
#             learnings=learnings
#         )
        
#         logger.info(f"Processing query: {query_context.query_id}")
#         logger.info(f"Feedback provided: {'Yes' if feedback else 'No'}")
#         logger.info(f"Filter values provided: {'Yes' if filter_values else 'No'}")
#         logger.info(f"Learnings provided: {'Yes' if learnings else 'No'}")

#         try:
#             # Step 1: Analyze intent and decompose, incorporating feedback, filter values, and learnings
#             print('-'*47)
#             print("Analyzing Intent and Decomposing the Problem (with feedback, filter values, and learnings)...")
#             analysis = self.intent_agent.analyze(query_context)
#             logger.info(f"Query decomposed into {len(analysis['sub_problems'])} sub-problems")

#             # Step 2: Process all sub-problems concurrently, with relevant feedback, filter values, and learnings
#             print('-'*47)
#             print("Processing the Sub-Problems (with feedback, filter values, and learnings)...")
#             solutions = await self.process_sub_problems(analysis['sub_problems'], query_context)
#             logger.info(f"Generated solutions for all sub-problems")

#             # Step 3: Integrate solutions
#             print('-'*47)
#             print("Integrating Solutions of Sub-Problems...")
#             final_result = self.integration_agent.integrate(solutions, query_context)
#             logger.info("Solutions integrated successfully")

#             print('-'*47)
#             print("Printing Final Result...")
#             print(json.dumps(final_result, indent=2))

#             return {
#                 "query_context": vars(query_context),
#                 "analysis": analysis,
#                 "solutions": solutions,
#                 "final_result": final_result
#             }

#         except Exception as e:
#             logger.error(f"Error processing query: {e}")
#             raise

# # Modified main function to demonstrate feedback, filter values, and learnings
# async def main():
#     _ = load_dotenv()
#     api_key = os.getenv("OPENAI_API_KEY")
#     processor = FilterAwareRecruitmentQueryProcessor(api_key)
    
#     # Example of filter values from a previous run
#     example_filter_values = {
#         "job_title/business_function": {
#             "current": [{"title_name": "Sales Executive", "min_staff": 0, "max_staff": 50000000}],
#             "past": [],
#             "both": [],
#             "event": "CURRENT"
#         },
#         "management_level": {
#             "current": [],
#             "past": [],
#             "both": [],
#             "event": ""
#         },
#         "location": {
#             "current": [],
#             "past": [],
#             "both": [],
#             "event": ""
#         }
#     }
    
#     queries = [
#         # {
#         #     "query": "Find sales executives.",
#         #     "feedback": "We need to focus on senior sales executives only.",
#         #     "filter_values": example_filter_values,
#         #     "learnings": ""
#         # },
#         # {
#         #     "query": "Find VP-level leaders from companies in Western Europe with over 30 years of experience and expertise in corporate finance.",
#         #     "feedback": "The results included too many people from Eastern Europe. Also, we need to see people who specifically have M&A experience.",
#         #     "filter_values": None,
#         #     "learnings": ""
#         # },
#         {
#             "query": "Find me a Chief Marketing Officer with a strong track record in scaling AI-driven SaaS platforms. Preferably based in the U.S.",
#             "feedback": "",  # "People who are not CMOs are appearing a lot. The results included too many people from US. Also, we need to see people who specifically have worked in sales in the past.",
#             "learnings": """- The company filter can only generate well-known companies
#                             - When finding profiles that identify people with strong track record in any field, the min company tenure should be 3
#                             - The search engine treats individual keywords independently and may not understand relationships between terms (e.g., "scaling AI platforms" vs just "AI" and "platforms")
#                             - Action verbs (like "scaling") may need to be emphasized more explicitly in profile_keywords to ensure relevant experience
#                             - Job titles are interpreted literally, so including broader titles (like VP when looking for C-level) will bring in unwanted profiles
#                             - The search engine doesn't automatically prioritize the most important aspects of a query - all keywords appear to have similar weight
#                             - When looking for specific experience (like "scaling"), it's better to be explicit in profile_keywords rather than relying on company_keywords alone
#                             - Broader search terms lead to more profiles but with lower precision - narrower terms yield fewer but more relevant results
#                             """,
#             "filter_values": None
#         },
#     ]
    
#     query_results = []
#     for query_item in queries:
#         print('-'*47)
#         print("QUERY: ", query_item["query"])
#         print("FEEDBACK: ", query_item["feedback"])
#         print("FILTER VALUES: ", "Provided" if query_item["filter_values"] else "None")
#         print("LEARNINGS: ", "Provided" if query_item.get("learnings") else "None")
        
#         try:
#             result = await processor.process_query_async(
#                 query_item["query"], 
#                 query_item["feedback"],
#                 query_item["filter_values"],
#                 query_item.get("learnings", "")
#             )
#             query_results.append({
#                 "query": query_item["query"], 
#                 "feedback": query_item["feedback"],
#                 "filter_values": query_item["filter_values"],
#                 "learnings": query_item.get("learnings", ""),
#                 "result": result
#             })
#         except Exception as e:
#             traceback.print_exc()
#             print(f"Error processing query: {e}")
    
#     return query_results

# # If you just want to test a single query with feedback and filter values
# async def test_single_query():
#     _ = load_dotenv()
#     api_key = os.getenv("OPENAI_API_KEY")
#     processor = FilterAwareRecruitmentQueryProcessor(api_key)
    
#     query = "Find VP-level leaders from AI companies in Western Europe with over 30 years of experience and expertise in corporate finance who have worked previously as Salesman."
#     feedback = "The results included too many professionals from big tech companies like Google and Microsoft. We need to focus on smaller AI startups and scale-ups. Also, 'salesman' is too junior - we need people who were previously in sales management roles, not individual contributors."
    
#     # Example filter values
#     filter_values = {
#         "job_title/business_function": {
#             "current": [{"title_name": "VP", "min_staff": 0, "max_staff": 50000000}],
#             "past": [{"title_name": "Salesman", "min_staff": 0, "max_staff": 50000000}],
#             "both": [],
#             "event": "CURRENT AND PAST"
#         },
#         "location": {
#             "current": [{"continent": "Europe", "region": "Western Europe"}],
#             "past": [],
#             "both": [],
#             "event": "CURRENT"
#         },
#         "total_working_years": {"min": 30, "max": null}
#     }
    
#     result = await processor.process_query_async(query, feedback, filter_values)
#     return result

# if __name__ == "__main__":
#     results = await (main())
#     # Or for testing single query:
#     # result = asyncio.run(test_single_query())
    
#     with open("k_filter_feedback_results.json", "w") as f:
#         json.dump(results, f, indent=2)

INFO:__main__:Processing query: query_1263552615401363675
INFO:__main__:Feedback provided: No
INFO:__main__:Filter values provided: No
INFO:__main__:Learnings provided: Yes


-----------------------------------------------
QUERY:  Find me a Chief Marketing Officer with a strong track record in scaling AI-driven SaaS platforms. Preferably based in the U.S.
FEEDBACK:  
FILTER VALUES:  None
LEARNINGS:  Provided
-----------------------------------------------
Analyzing Intent and Decomposing the Problem (with feedback, filter values, and learnings)...
[
  {
    "role": "system",
    "content": "You are an expert recruitment query analyzer specializing in breaking down complex queries into ATOMIC logical sub-problems while prioritizing feedback incorporation and considering existing filter values.\n\n        Your PRIMARY RESPONSIBILITIES (in order of importance):\n        1. FEEDBACK FOCUS: Deeply analyze any provided feedback and make it the central guiding principle of your analysis\n        2. ATOMICITY: Break down queries into truly atomic sub-problems (each addressing ONE specific aspect only)\n        3. FILTER AWARENESS: Carefully consider existing filter

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:__main__:Query decomposed into 4 sub-problems


```json
{
    "main_intent": "Identify a Chief Marketing Officer with experience in scaling AI-driven SaaS platforms, preferably based in the U.S.",
    "feedback_analysis": "No specific feedback provided; focus is on breaking down the query into its atomic components as guided by system learnings.",
    "filter_analysis": "The existing filter for company tenure should be set at a minimum of 3 to ensure a strong track record. The location preference for 'U.S.' should be respected as a key constraint.",
    "learnings_analysis": "Several system learnings are directly applicable: emphasize 'scaling' in profile keywords, ensure job titles are specific to avoid broader unwanted titles, and use narrower terms for higher precision. The independent treatment of keywords by the search engine necessitates a clear and explicit listing of required experiences and skills.",
    "thinking": "The query was dissected into atomic sub-problems, each addressing a single element of the requirement, ensur

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:__main__:Generated solutions for all sub-problems


-----------------------------------------------
Integrating Solutions of Sub-Problems...
###############################################
[{'sub_problem_analysis': {'original_context': 'Find me a Chief Marketing Officer with a strong track record in scaling AI-driven SaaS platforms. Preferably based in the U.S.', 'thought_process': "The task is to narrow down individuals with the specific job title of 'Chief Marketing Officer'. The focus should be on literal interpretation of the job title, avoiding broader titles like VP or Director to prevent including unrelated profiles.", 'interpreted_requirement': "Accurately filter profiles to include only those with the job title 'Chief Marketing Officer'.", 'relevant_filters': {'identified_filters': ['JOB_TITLE/BUSINESS_FUNCTION'], 'rationale': "The job title filter will be used to ensure that only profiles with the exact title of 'Chief Marketing Officer' are considered."}, 'relevant_learnings': {'applicable_learnings': ['Job titles are interpr

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:__main__:Solutions integrated successfully


### 1. DETAILED ANALYSIS

**Solution 1: Job Title/Business Function**
- **Mapping to filter structure**: This solution requires using the "job_title/business_function" filter to target the exact job title "Chief Marketing Officer" without deviation.
- **Explicit vs Implicit Requirements**: Explicitly requires the exact job title; implicitly assumes no equivalent roles are necessary.
- **Temporal Aspects**: Focuses on the current position as the primary interest.
- **Logical Dependencies**: This filter relies on literal title interpretation to ensure precision.
- **Potential Conflicts**: None, as it strictly adheres to the requirement.
- **Feedback Influence**: No feedback provided, maintaining strict adherence to the literal title.
- **Existing Filter Influence**: Existing filter values favor precise title specifications.

**Solution 2: Experience in Scaling AI-driven SaaS Platforms**
- **Mapping to filter structure**: This maps to the "skills" filter, incorporating keywords related to

## Self Reflection incorporated approach

In [ ]:
from typing import Dict, List, Any, Optional
from dataclasses import dataclass
import openai
import json
from enum import Enum
from datetime import datetime
import logging
from dotenv import load_dotenv
import os
from openai import OpenAI
import ast
from pprint import pprint
from openai import AsyncOpenAI
import asyncio
import traceback
import re

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

@dataclass
class QueryContext:
    original_query: str
    timestamp: str
    query_id: str
    feedback: Optional[str] = None
    learnings: Optional[str] = None
    filter_values: Optional[Dict[str, Any]] = None  # Added filter_values field
    
@dataclass
class SubProblem:
    category: str
    description: str
    dependencies: List[str]
    priority: int
    context_reference: Dict[str, Any]
    query_segment: str
    relevant_feedback: Optional[str] = None
    relevant_learnings: Optional[str] = None
    relevant_filters: Optional[Dict[str, Any]] = None  # Added relevant_filters field

@dataclass
class Solution:
    sub_problem: SubProblem
    proposed_solution: Dict
    confidence: float
    reasoning: str
    validation_notes: List[str]

class Agent:
    def __init__(self, api_key: str):
        self.client = OpenAI(api_key=api_key)
        self.async_client = AsyncOpenAI(api_key=api_key)
        
    def call_llm(self, messages: List[Dict]) -> str:
        try:
            print(json.dumps(messages, indent=2))
            response = self.client.chat.completions.create(
                model="gpt-4o",
                messages=messages,
                temperature=0.7
            )
            print((response.choices[0].message.content))
            return response.choices[0].message.content
        except Exception as e:
            logger.error(f"Error in LLM call: {e}")
            raise

    async def call_llm_async(self, messages: List[Dict]) -> str:
        try:
            response = await self.async_client.chat.completions.create(
                model="gpt-4o",
                messages=messages,
                temperature=0.7
            )
            return response.choices[0].message.content
        except Exception as e:
            logger.error(f"Error in async LLM call: {e}")
            raise


class FilterAwareIntentAnalysisAgent(Agent):
    def __init__(self, api_key: str):
        super().__init__(api_key)
        self.system_prompt = """You are an expert recruitment query analyzer specializing in breaking down complex queries into ATOMIC logical sub-problems while prioritizing feedback incorporation and considering existing filter values.

        Your PRIMARY RESPONSIBILITIES (in order of importance):
        1. FEEDBACK FOCUS: Deeply analyze any provided feedback and make it the central guiding principle of your analysis
        2. ATOMICITY: Break down queries into truly atomic sub-problems (each addressing ONE specific aspect only)
        3. FILTER AWARENESS: Carefully consider existing filter values when analyzing the query
        4. LEARNING INTEGRATION: Apply relevant system learnings to each atomic sub-problem
        5. INTENT UNDERSTANDING: Comprehend the main intent of the recruitment query
        
        Key principles for FEEDBACK incorporation:
        - Treat feedback as the highest priority guidance for your analysis
        - Explicitly connect each piece of feedback to specific atomic sub-problems
        - If feedback contradicts the original query, prioritize the feedback's direction
        - Use feedback to refine, modify, or completely change your understanding of requirements
        - For each atomic sub-problem, include SPECIFIC quotes from the feedback that are relevant
        
        Key principles for ATOMIC SUB-PROBLEM creation:
        - Each sub-problem must address EXACTLY ONE specific requirement or concept
        - If a potential sub-problem addresses multiple requirements, split it further
        - Test each sub-problem with the question: "Does this solve exactly one thing?"
        - Avoid compound requirements within a single sub-problem
        - Ensure each sub-problem has a singular, clear focus
        
        Key principles for LEARNING INTEGRATION:
        - For each sub-problem, identify SPECIFIC relevant learnings that apply
        - Only associate learnings that directly inform or constrain the specific sub-problem
        - Extract only the portions of learnings that are relevant to each atomic element
        - Prioritize learnings that address similar requirements to the current sub-problem
        - Connect each learning to the specific reasoning it enables for the sub-problem
        
        For each ATOMIC sub-problem, you must:
        1. Focus on a single, specific aspect of the requirement
        2. Connect relevant feedback specifically to this atomic element
        3. Identify which specific filter values (if any) relate to this single element
        4. Extract relevant system learnings specifically applicable to this element
        5. Assign appropriate priority (1-5, 1 being highest)
        6. Explicitly state dependencies on other atomic sub-problems
        
        Your analysis process should follow these steps:
        1. First, thoroughly analyze the feedback to identify key guidance points
        2. Second, examine existing filter values to understand constraints
        3. Third, review system learnings to identify relevant past solutions
        4. Fourth, break down the query into the smallest possible atomic sub-problems
        5. Finally, connect feedback points, filter values, and system learnings to each atomic sub-problem
        
        Output JSON structure:
        {
            "main_intent": "Clear statement of primary goal",
            "feedback_analysis": "COMPREHENSIVE analysis of how feedback modifies the query - this should be DETAILED",
            "filter_analysis": "Analysis of how existing filter values constrain the solution",
            "learnings_analysis": "Analysis of how system learnings inform the solution approach",
            "thinking": "Step-by-step breakdown of how you arrived at your atomic sub-problems",
            "analysis_notes": ["Detailed thoughts and reasoning"],
            "potential_challenges": ["Identified challenges"],
            "sub_problems": [
                {
                    "category": "job_title/location/education/etc",
                    "description": "Description of this SINGLE atomic requirement",
                    "dependencies": ["other_category_ids"],
                    "priority": "1-5",
                    "context_reference": {"query_segment": "relevant part of original query"},
                    "query_segment": "exact text from original query",
                    "relevant_feedback": "SPECIFIC feedback directly relevant to this atomic sub-problem",
                    "relevant_filters": {"filter_name": "filter_value"},
                    "relevant_learnings": ["SPECIFIC learnings directly applicable to this atomic sub-problem"]
                }
            ],
            "confidence_score": 0.0
        }"""

    def analyze(self, query_context: QueryContext) -> Dict:
        feedback_section = ""
        if query_context.feedback:
            feedback_section = f"""
            CRITICAL FEEDBACK INFORMATION:
            The following feedback has been provided about previous attempts to solve this query:
            {query_context.feedback}
            
            THIS FEEDBACK IS YOUR PRIMARY GUIDANCE. You must:
            1. Make this feedback the central focus of your analysis
            2. For each atomic sub-problem, identify specific parts of the feedback that apply to it
            3. Use feedback to reinterpret or modify the original query when needed
            4. If feedback conflicts with the original query, prioritize the feedback's direction
            5. Extract specific quotes from the feedback for each atomic sub-problem
            """
        
        filter_section = ""
        if query_context.filter_values:
            filter_section = f"""
            EXISTING FILTER VALUES:
            {json.dumps(query_context.filter_values, indent=2)}
            
            Consider these filter values when creating atomic sub-problems:
            1. For each atomic sub-problem, identify specific relevant filter values
            2. Understand how each filter constrains or guides that atomic requirement
            3. Note where existing filters might already address parts of the query
            """
        
        learnings_section = ""
        if hasattr(query_context, 'learnings') and query_context.learnings:
            learnings_section = f"""
            SYSTEM LEARNINGS:
            {query_context.learnings}
            
            Apply these learnings to your analysis:
            1. For each atomic sub-problem, identify SPECIFIC relevant learnings
            2. Extract only the portions of learnings that apply to each sub-problem
            3. Use learnings to inform how you break down and categorize requirements
            4. Connect specific learnings to specific sub-problems
            5. Include direct references to relevant learnings for each sub-problem
            """
        
        prompt = f"""Analyze this recruitment query, focusing on breaking it down into TRULY ATOMIC sub-problems: 
        {query_context.original_query}

        {feedback_section}
        
        {filter_section}
        
        {learnings_section}

        IMPORTANT ANALYSIS INSTRUCTIONS:
        1. PRIMARILY focus on the feedback (if provided) to guide your understanding
        2. Break the query into the SMALLEST POSSIBLE atomic sub-problems (each addressing EXACTLY ONE thing)
        3. Test each sub-problem by asking: "Is this addressing exactly one specific requirement?"
        4. If a sub-problem addresses multiple things, split it further
        5. Connect each piece of feedback to specific atomic sub-problems using direct quotes
        6. Match relevant filter values to each atomic sub-problem
        7. Connect specific system learnings to each relevant atomic sub-problem
        
        Show your detailed reasoning process, highlighting how you:
        1. Interpreted the feedback
        2. Broke the query into truly atomic sub-problems
        3. Connected feedback to each atomic element
        4. Considered existing filter values in your analysis
        5. Applied system learnings to inform your sub-problem identification
        """

        messages = [
            {"role": "system", "content": self.system_prompt},
            {"role": "user", "content": prompt}
        ]
        
        response = self.call_llm(messages)
        try:
            response = eval(response[response.find('{'):response.rfind('}') + 1])
            return response
        except Exception as e:
            logger.error(f"Error parsing intent analysis response: {e}")
            # Fallback to trying json.loads
            try:
                json_str = response[response.find('{'):response.rfind('}') + 1]
                return json.loads(json_str)
            except Exception as e2:
                logger.error(f"Second error parsing intent analysis: {e2}")
                raise ValueError(f"Failed to parse intent analysis response: {e}, {e2}")
    

class FilterAwareSubProblemAnalysisAgent(Agent):
    def __init__(self, api_key: str):
        super().__init__(api_key)
        self.system_prompt = """You are an expert in solving specific recruitment sub-problems, with the ability to incorporate targeted feedback, existing filter values, and system learnings.
        IMPORTANT: Focus ONLY on the specific sub-problem assigned to you. While you may reference
        the original query for context, your solution should be specifically targeted to your assigned sub-problem.

        ## FILTER SYSTEM REFERENCE GUIDE
        This comprehensive guide explains all available filters and their proper usage:


        1. SKILL/KEYWORDS
        - Purpose: Match profiles containing specific skills, expertise, or specializations
        - Format: List of strings (e.g., ["Python", "project management", "recruiting"])
        - Usage: Profiles must contain at least ONE of the listed skills/keywords
        - Example: {"SKILL": ["Python", "machine learning", "data science"]}

        2. INDUSTRY
        - Purpose: Filter profiles by industry experience
        - Format: List of strings (e.g., ["Finance", "Technology", "Healthcare"])
        - Usage: Displays profiles with experience in any of the specified industries
        - Example: {"INDUSTRY": ["Healthcare", "Pharmaceuticals"]}

        3. COMPANY
        - Purpose: Filter by current or past company association
        - Format: List of strings representing companies or company characteristics
        - Usage: Identifies relevant companies based on extracted information
        - Example: {"COMPANY": ["Microsoft and similar companies", "companies based in USA"]}

        4. LOCATION
        - Purpose: Filter profiles by geographic location
        - Format: List of strings (e.g., ["New York", "Europe", "Countries near Egypt"])
        - Usage: Identifies specific locations or regions, including nearby areas when specified
        - Example: {"LOCATION": ["San Francisco", "Bay Area"]}

        5. EDUCATION
        - Purpose: Filter by academic qualifications
        - Format: List of dictionaries with degree and optional major
        - Usage: Matches profiles with specific educational backgrounds
        - Example: {"EDUCATION": [{"degree": "Master's", "major": "Business Administration"}]}

        6. NAME
        - Purpose: Search for specific individuals by name
        - Format: List of strings (e.g., ["John Doe", "Will Smith"])
        - Usage: Only returns profiles with exact name matches
        - Example: {"NAME": ["John Smith", "Maria Garcia"]}

        7. SCHOOL
        - Purpose: Filter by educational institution
        - Format: List of strings (e.g., ["Stanford University", "Yale University"])
        - Usage: Profiles must be associated with at least one of the specified schools
        - Example: {"SCHOOL": ["Harvard University", "MIT"]}

        8. COMPANY_TENURE
        - Purpose: Filter by length of time in CURRENT company or industry
        - Format: String representing duration (e.g., "2 years", "5+ years")
        - Usage: Only shows profiles with specified tenure in their current company
        - Example: {"COMPANY_TENURE": "3+ years"}

        9. ROLE_TENURE
        - Purpose: Filter by length of time in CURRENT role or position
        - Format: String representing duration (e.g., "2 years", "5+ years")
        - Usage: Only shows profiles with specified tenure in their current role
        - Example: {"ROLE_TENURE": "2+ years"}

        10. TOTAL_WORKING_YEARS
        - Purpose: Filter by overall career duration
        - Format: List of two integers representing minimum and maximum years
        - Usage: Shows profiles with total experience within the specified range
        - Example: {"TOTAL_WORKING_YEARS": [5, 10]}

        11. GENDER
        - Purpose: Filter profiles by gender
        - Format: List containing "Female" if female-only search is required
        - Usage: When specified as ["Female"], only female profiles are shown
        - Example: {"GENDER": ["Female"]}

        12. AGE
        - Purpose: Filter profiles by age category
        - Format: List of strings from predefined categories: "Under 25", "Over 50", "Over 65"
        - Usage: Shows profiles matching the specified age categories
        - Example: {"AGE": ["Over 50"]}

        13. ETHNICITY
        - Purpose: Filter profiles by ethnic background
        - Format: List of strings from predefined ethnicities: "Asian", "African", "Hispanic", etc.
        - Usage: Only applied when explicitly mentioned in the context of ethnicity
        - Example: {"ETHNICITY": ["South Asian", "Hispanic"]}

        14. CURRENT_OWNERSHIP
        - Purpose: Filter by company ownership type
        - Format: List of strings from: "Public", "Private", "VC Funded", "Private Equity Backed"
        - Usage: Only applied when explicitly mentioned; shows profiles at companies with that ownership
        - Example: {"CURRENT_OWNERSHIP": ["VC Funded"]}

        15. JOB_TITLE/BUSINESS_FUNCTION
        - Purpose: Filter by job roles or business functions
        - Format: List of strings representing job titles or functions
        - Usage: Matches specific roles or entire functional areas
        - Example: {"JOB_TITLE": ["Chief Sales Officer", "Sales Director"]}

        16. MANAGEMENT_LEVEL
        - Purpose: Filter by management hierarchy
        - Format: List of strings from predefined levels like "C-Suite/Chiefs", "Director", etc.
        - Usage: Only extracted when user asks for entire management domains
        - Example: {"MANAGEMENT_LEVEL": ["C-Suite/Chiefs", "Executive VP or Sr. VP"]}


        For this specific sub-problem, follow this detailed analysis process:
        1. Initial Assessment
           - Understand the specific requirement of THIS sub-problem
           - Review any specific feedback provided for this sub-problem
           - Review any existing filter values relevant to this sub-problem
           - Review relevant system learnings applicable to this sub-problem
           - Identify relevant filter categories for THIS component
           - Review dependencies and constraints specific to THIS aspect

        2. Feedback, Filter, and Learning Integration
           - Carefully analyze how the provided feedback applies to this specific sub-problem
           - Consider how existing filter values constrain or guide your solution
           - Apply specific insights from system learnings to this sub-problem
           - Adjust your approach based on the feedback, filter values, and learnings
           - Address any issues or improvements mentioned in the feedback
           - Build upon or refine existing filter values
           - Draw inspiration from system learnings when proposing solutions

        3. Solution Generation
           - FILTER-FIRST APPROACH: Each solution MUST manipulate specific filters
           - Try to generate 3 proposed solution for THIS specific sub-problem
           - For each solution, identify exactly which filters to use and what values to assign
           - ALL solutions must involve creating, modifying, or refining filter values
           - Be precise about filter values - use exact formats as shown in the reference guide
           - Generate multiple filter-based approaches for THIS specific requirement
           - Take direct inspiration from system learnings when formulating solutions
           - Consider edge cases and limitations
           - Ensure solution stays focused on the assigned sub-problem
           - Apply the lessons learned from feedback
           - Consider compatibility with existing filter values
        
        4. Solution Evaluation
           - Score each filter-based solution (1-10)
           - List pros and cons specific to this filter approach
           - Consider implementation feasibility of the filter configuration
           - Evaluate how well the filter solution addresses the feedback
           - Evaluate how well the filter solution integrates with existing filter values
           - Assess how well the solution applies insights from system learnings

        5. Final Selection
           - Choose optimal filter-based solution for this specific component
           - Document reasoning for this specific filter configuration
           - Provide confidence score for the selected filter approach
           - Explain how feedback influenced your final filter choice
           - Explain how existing filter values influenced your final filter configuration
           - Explain how system learnings influenced your solution selection

        CRITICAL: Every proposed solution MUST involve manipulating one or more filters. Solutions that don't directly specify filter actions are invalid.

        Remember: Stay focused on your assigned sub-problem. Don't try to solve the entire query.

        Output structure:
        {
            "sub_problem_analysis": {
                "original_context": "",
                "thought_process": "",
                "interpreted_requirement": "",
                "relevant_filters": {
                    "identified_filters": [],
                    "rationale": ""
                },
                "relevant_learnings": {
                    "applicable_learnings": [],
                    "learning_insights": ""
                },
                "filter_integration": "",
                "feedback_integration": "",
                "learning_integration": "",
                "proposed_solutions": [
                    {
                        "thinking": "",
                        "filter_actions": {
                            "filter_name": "description of action with specific values"
                        },
                        "feedback_addressed": "",
                        "learning_applied": "",
                        "implementation_notes": "",
                        "approach": "",
                        "score": 0,
                        "pros": [],
                        "cons": [],
                        "filter_compatibility": ""
                    }
                ],
                "selected_solution": {
                    "thinking_for_selection": "",
                    "filter_values": {
                        "filter_name": "specific_value"
                    },
                    "confidence": 0.0,
                    "reasoning": "",
                    "validation_notes": [],
                    "feedback_influence": "",
                    "filter_influence": "",
                    "learning_influence": ""
                }
            }
        }"""
        
        self.reflection_prompt = """You are an expert self-critical evaluator of recruitment filter solutions. Your task is to evaluate your own solution for a specific sub-problem and identify any issues, improvements, or inconsistencies.

        Critically evaluate your proposed solution for the sub-problem, considering:

        1. SUB-PROBLEM ALIGNMENT:
           - Does the solution directly address the exact requirements of the sub-problem?
           - Are you solving only what was asked for in this specific sub-problem?
           - Are you staying within the scope of this specific sub-problem?

        2. FILTER SELECTION AND FORMAT:
           - Are you using the most appropriate filters for this requirement?
           - Are your filter values in the correct format according to the reference guide?
           - Are there better filter combinations you could have used?
           - Are your filter values specific and precise enough?

        3. FEEDBACK INCORPORATION:
           - Have you fully addressed all aspects of the relevant feedback?
           - Is your solution consistent with the direction indicated in the feedback?
           - Have you properly prioritized feedback over original requirements if they conflict?

        4. EXISTING FILTER INTEGRATION:
           - Does your solution build upon and enhance existing filter values?
           - Are there any conflicts with existing filter configurations?
           - Have you properly considered how your changes will work with existing filters?

        5. LEARNING APPLICATION:
           - Have you fully applied relevant learnings to inform your solution?
           - Are you missing opportunities to use insights from system learnings?
           - Does your solution reflect best practices from previous similar cases?

        6. SOLUTION PRACTICALITY:
           - Is your solution implementation realistic and feasible?
           - Have you considered potential edge cases and limitations?
           - Is your confidence score justified by your reasoning?

        Be honest, specific, and constructive. Identify concrete issues and suggest specific improvements.

        Output format:
        {
            "requires_iteration": true/false,
            "issues": {
                "sub_problem_alignment": ["specific issue description"],
                "filter_selection": ["specific issue description"],
                "feedback_incorporation": ["specific issue description"],
                "filter_integration": ["specific issue description"],
                "learning_application": ["specific issue description"],
                "solution_practicality": ["specific issue description"]
            },
            "improvement_suggestions": ["specific actionable suggestion"],
            "critical_gaps": ["most important issues to address"],
            "strengths": ["what was done well"],
            "overall_assessment": "Summary assessment"
        }"""

    async def solve_async(self, sub_problem: SubProblem, query_context: QueryContext) -> Solution:
        feedback_section = ""
        if sub_problem.relevant_feedback:
            feedback_section = f"""
            The following feedback is specifically relevant to this sub-problem:
            {sub_problem.relevant_feedback}
            
            Make sure your solution directly addresses this feedback.
            """
        elif query_context.feedback:
            feedback_section = f"""
            General feedback on the query (consider only what's relevant to this sub-problem):
            {query_context.feedback}
            """

        filter_section = ""
        if sub_problem.relevant_filters:
            filter_section = f"""
            The following filter values are specifically relevant to this sub-problem:
            {json.dumps(sub_problem.relevant_filters, indent=2)}
            
            Consider these filter values when developing your solution.
            You can build upon, refine, or modify these filter values as needed.
            """
        elif query_context.filter_values:
            filter_section = f"""
            General filter values from previous processing (consider only what's relevant to this sub-problem):
            {json.dumps(query_context.filter_values, indent=2)}
            """
        
        learnings_section = ""
        if hasattr(sub_problem, 'relevant_learnings') and sub_problem.relevant_learnings:
            learnings_section = f"""
            The following system learnings are specifically relevant to this sub-problem:
            {sub_problem.relevant_learnings}
            
            Use these specific learnings to inform your solution approach.
            Draw direct inspiration from these learnings when proposing and selecting filter actions.
            """
        elif hasattr(query_context, 'learnings') and query_context.learnings:
            learnings_section = f"""
            General system learnings (consider only what's relevant to this sub-problem):
            {query_context.learnings}
            
            Extract insights relevant to this specific sub-problem to inform your solutions.
            """

        prompt = f"""Focus on solving this specific sub-problem:

        Sub-problem Category: {sub_problem.category}
        Sub-problem Description: {sub_problem.description}
        Specific Query Segment to Address: {sub_problem.query_segment}

        {feedback_section}
        
        {filter_section}
        
        {learnings_section}

        For reference only, original query context: {query_context.original_query}

        IMPORTANT INSTRUCTIONS:
        1. Your solution MUST involve manipulating specific filters
        2. Each proposed solution must specify exactly which filters to create/modify/refine
        3. Be precise about filter values and formats according to the reference guide
        4. Every solution must result in concrete filter values
        5. Don't just describe what to do - specify exact filter configurations
        6. Draw direct inspiration from relevant system learnings when designing solutions

        Remember: Focus only on solving this specific sub-problem component.
        Do not attempt to solve the entire query.

        Show your detailed reasoning process for this specific component,
        including how you've incorporated the relevant feedback, filter values, and system learnings."""

        messages = [
            {"role": "system", "content": self.system_prompt},
            {"role": "user", "content": prompt}
        ]
        
        # Get initial solution
        response = await self.call_llm_async(messages)
        initial_solution_text = response
        
        try:
            # Extract JSON from the response
            initial_solution = json.loads(response[response.find('{'):response.rfind('}') + 1])
            
            # Make sure we store the sub-problem category for easy reference later
            if "sub_problem_analysis" in initial_solution:
                # Store the category directly in the solution for easier extraction later
                if "original_context" not in initial_solution["sub_problem_analysis"]:
                    initial_solution["sub_problem_analysis"]["original_context"] = ""
                
                initial_solution["sub_problem_analysis"]["category"] = sub_problem.category
                initial_solution["sub_problem_analysis"]["original_context"] = f"Sub-problem Category: {sub_problem.category}\nSub-problem Description: {sub_problem.description}\nSpecific Query Segment to Address: {sub_problem.query_segment}"
            
            # Self-reflection step
            reflection_prompt = f"""Now, critically evaluate your own solution for this sub-problem:

            Sub-problem Category: {sub_problem.category}
            Sub-problem Description: {sub_problem.description}
            Specific Query Segment to Address: {sub_problem.query_segment}

            Your proposed solution:
            {json.dumps(initial_solution, indent=2)}

            Carefully examine your solution for any issues, gaps, or areas for improvement.
            Be honest and critical - identify any weaknesses or potential problems.
            """

            reflection_messages = [
                {"role": "system", "content": self.reflection_prompt},
                {"role": "user", "content": reflection_prompt}
            ]
            
            # Get self-reflection
            reflection_response = await self.call_llm_async(reflection_messages)
            try:
                reflection_json = json.loads(reflection_response[reflection_response.find('{'):reflection_response.rfind('}') + 1])
            except Exception as e:
                logger.warning(f"Error parsing reflection response: {e}. Using default reflection.")
                reflection_json = {
                    "requires_iteration": False,
                    "issues": {},
                    "improvement_suggestions": ["Couldn't parse reflection properly"],
                    "critical_gaps": [],
                    "strengths": ["Original solution maintained"],
                    "overall_assessment": "Reflection parsing failed, using original solution"
                }
            
            # Determine if iteration is needed based on reflection
            if reflection_json.get("requires_iteration", False):
                # Create a refinement prompt based on the reflection
                refinement_prompt = f"""I need you to revise and improve your solution for this sub-problem:

                Sub-problem Category: {sub_problem.category}
                Sub-problem Description: {sub_problem.description}
                Specific Query Segment to Address: {sub_problem.query_segment}

                {feedback_section}
                
                {filter_section}
                
                {learnings_section}

                Your previous solution had these issues:
                {json.dumps(reflection_json.get("issues", {}), indent=2)}

                Critical gaps to address:
                {json.dumps(reflection_json.get("critical_gaps", []), indent=2)}

                Suggested improvements:
                {json.dumps(reflection_json.get("improvement_suggestions", []), indent=2)}

                Please provide an improved solution that addresses these issues and incorporates the suggested improvements.
                Be sure to specifically resolve the critical gaps identified.
                
                Your original solution for reference:
                {json.dumps(initial_solution, indent=2)}
                """

                refinement_messages = [
                    {"role": "system", "content": self.system_prompt},
                    {"role": "user", "content": refinement_prompt}
                ]
                
                # Get refined solution
                refined_response = await self.call_llm_async(refinement_messages)
                try:
                    refined_solution = json.loads(refined_response[refined_response.find('{'):refined_response.rfind('}') + 1])
                    
                    # Make sure we store the sub-problem category in the refined solution as well
                    if "sub_problem_analysis" in refined_solution:
                        refined_solution["sub_problem_analysis"]["category"] = sub_problem.category
                        if "original_context" not in refined_solution["sub_problem_analysis"]:
                            refined_solution["sub_problem_analysis"]["original_context"] = f"Sub-problem Category: {sub_problem.category}\nSub-problem Description: {sub_problem.description}\nSpecific Query Segment to Address: {sub_problem.query_segment}"
                    
                    # Add self-reflection metadata to the solution
                    refined_solution["sub_problem_analysis"]["self_reflection"] = {
                        "initial_assessment": reflection_json,
                        "required_iteration": True,
                        "improvement_areas": reflection_json.get("critical_gaps", []),
                        "refinement_process": "Solution was refined based on self-reflection"
                    }
                    
                    return refined_solution
                except Exception as e:
                    logger.error(f"Error parsing refined solution: {e}. Falling back to original solution.")
                    # If parsing fails, add reflection to original and return that
                    initial_solution["sub_problem_analysis"]["self_reflection"] = {
                        "assessment": reflection_json,
                        "required_iteration": True,
                        "refinement_error": str(e),
                        "conclusion": "Refinement attempted but failed, using original solution"
                    }
                    return initial_solution
            else:
                # If no iteration required, just add reflection metadata to original solution
                initial_solution["sub_problem_analysis"]["self_reflection"] = {
                    "assessment": reflection_json,
                    "required_iteration": False,
                    "strengths": reflection_json.get("strengths", []),
                    "conclusion": "Solution was determined to be sufficient without refinement"
                }
                
                return initial_solution
                
        except Exception as e:
            logger.error(f"Error in solution processing or self-reflection: {e}")
            traceback.print_exc()
            # Fallback to returning a structured solution with the error
            try:
                solution_json = json.loads(initial_solution_text[initial_solution_text.find('{'):initial_solution_text.rfind('}') + 1])
                # Add category and error information
                if "sub_problem_analysis" in solution_json:
                    solution_json["sub_problem_analysis"]["category"] = sub_problem.category
                    solution_json["sub_problem_analysis"]["error_during_processing"] = str(e)
                return solution_json
            except:
                # If even that fails, return a structured error
                return {
                    "sub_problem_analysis": {
                        "category": sub_problem.category,
                        "error": f"Error processing solution: {str(e)}",
                        "interpreted_requirement": sub_problem.description
                    }
                }


class SolutionReflectionAgent(Agent):
    def __init__(self, api_key: str):
        super().__init__(api_key)
        self.system_prompt = """You are an expert evaluator of recruitment filter solutions. Your role is to analyze a set of solutions and identify any that require further iteration or improvement.

        For each sub-problem solution, you will:
        1. Evaluate the overall quality and completeness
        2. Identify any logical inconsistencies or errors
        3. Assess whether the solution properly addresses the sub-problem
        4. Determine if feedback was properly incorporated 
        5. Check if filter values are correctly formatted and appropriate
        6. Verify that relevant learnings were applied
        7. Identify solutions that need further refinement

        Look specifically for:
        - Solutions that don't properly address the sub-problem's core requirement
        - Inconsistencies between the sub-problem and the solution approach
        - Filter values that are incorrectly formatted or inappropriate
        - Missing incorporation of relevant feedback
        - Failure to apply relevant system learnings
        - Logical contradictions within the solution
        - Unclear or ambiguous filter specifications
        - Solutions with low confidence scores without proper justification

        When identifying issues, be specific about what needs to be improved and why.

        Output structure:
        {
            "overall_assessment": "Overall evaluation of all solutions",
            "solutions_requiring_iteration": [
                {
                    "sub_problem_category": "Category of the sub-problem",
                    "sub_problem_description": "Description of the sub-problem",
                    "issues": ["Specific issues identified"],
                    "suggested_improvements": ["Specific improvement suggestions"],
                    "feedback_for_agent": "Detailed guidance for improving this solution"
                }
            ],
            "solutions_approved": ["List of sub-problem categories that are acceptable"],
            "quality_issues": ["Any overall quality concerns across all solutions"]
        }"""

    async def evaluate_solutions(self, solutions: List[Dict], query_context: QueryContext) -> Dict:
        prompt = f"""Evaluate the following set of sub-problem solutions for a recruitment query:

        Original Query: {query_context.original_query}
        
        Feedback (if any): {query_context.feedback if query_context.feedback else "None provided"}
        
        Existing Filter Values (if any): {json.dumps(query_context.filter_values, indent=2) if query_context.filter_values else "None provided"}
        
        System Learnings (if any): {query_context.learnings if hasattr(query_context, 'learnings') and query_context.learnings else "None provided"}

        Number of Solutions to Evaluate: {len(solutions)}

        Solutions:
        {json.dumps(solutions, indent=2)}

        For each solution:
        1. Evaluate if it properly addresses its assigned sub-problem
        2. Check if it appropriately incorporates relevant feedback
        3. Verify that filter values are correctly formatted and appropriate
        4. Assess if it properly applies relevant system learnings
        5. Identify any logical inconsistencies or errors
        
        Identify any solutions that require further iteration and provide specific feedback on how they should be improved.
        Be sure to properly extract the sub-problem category from each solution.
        """

        messages = [
            {"role": "system", "content": self.system_prompt},
            {"role": "user", "content": prompt}
        ]
        
        response = await self.call_llm_async(messages)
        
        try:
            evaluation = json.loads(response[response.find('{'):response.rfind('}') + 1])
            return evaluation
        except Exception as e:
            logger.error(f"Error parsing solution evaluation: {e}")
            return {
                "error": str(e),
                "raw_response": response,
                "solutions_requiring_iteration": []
            }


class IntegrationAgent(Agent):
    def __init__(self, api_key: str):
        super().__init__(api_key)
        self.system_prompt = """You are an expert in integrating and validating recruitment filter solutions and converting them into specific JSON filter formats.

        Your critical responsibilities:
        1. Analyze all solutions thoroughly
        2. Provide detailed reasoning for each decision
        3. Convert the solutions into the exact filter format
        4. Only include relevant filters with valid values
        5. Ensure all values are logically consistent
        6. Consider feedback and existing filter values in your integration

        The final output must strictly follow this filter structure:
        {
            "job_title/business_function": {
                "current": [{"title_name": "", "min_staff": 0, "max_staff": 50000000}],
                "past": [{"title_name": "", "min_staff": 0, "max_staff": 50000000}],
                "both": [{"title_name": "", "min_staff": 0, "max_staff": 50000000}],
                "event": ""  # Can only be "CURRENT", "PAST", "CURRENT OR PAST", "CURRENT AND PAST"
            },
            "management_level": {
                "current": [],
                "past": [],
                "both": [],
                "event": ""
            },
            "location": {
                "current": [],
                "past": [],
                "both": [],
                "event": ""
            },
            "company": {
                "current_prompt": "",
                "past_prompt": "",
                "event": ""
            },
            "education": [{"degree": "", "major": ""}],
            "school": [],
            "current_ownership": [],
            "name": [],
            "ethnicity": [],
            "age": [],
            "total_working_years": {"min": null, "max": null},
            "role_tenure": {"min": null, "max": null},
            "company_tenure": {"min": null, "max": null},
            "skills": []
        }

        Before providing the final JSON:
        1. Analyze each requirement thoroughly
        2. Explain your reasoning for each filter value
        3. Discuss any assumptions or implicit requirements
        4. Validate logical consistency
        5. Only include filters that have valid values
        6. Explain any complex decisions or trade-offs
        7. Discuss how feedback has been addressed
        8. Explain how existing filter values have been incorporated

        Your response should follow this structure:
        1. DETAILED ANALYSIS
        2. REASONING FOR EACH FILTER
        3. ASSUMPTIONS AND IMPLICATIONS
        4. VALIDATION CHECKS
        5. FEEDBACK INCORPORATION
        6. FINAL FILTER JSON
        """

    def integrate(self, solutions: List[Solution], query_context: QueryContext) -> Dict:
        print("#"*47)
        print(solutions)
        print("#"*47)
        
        feedback_section = ""
        if query_context.feedback:
            feedback_section = f"""
            Previous feedback on the query solution:
            {query_context.feedback}
            
            Ensure your integration addresses this feedback.
            """
            
        filter_section = ""
        if query_context.filter_values:
            filter_section = f"""
            Existing filter values from previous processing:
            {json.dumps(query_context.filter_values, indent=2)}
            
            Consider these filter values in your integration.
            You can build upon, refine, or modify these filter values as needed.
            """
            
        prompt = f"""Integrate these solutions while maintaining alignment with the original query:

        Original Query: {query_context.original_query}

        {feedback_section}
        
        {filter_section}

        Please provide:
        1. Detailed analysis of how each solution component fits into the final filter structure
        2. Explicit reasoning for each filter value being set
        3. Discussion of any assumptions or implicit requirements
        4. Explanation of any trade-offs or decisions made
        5. Explanation of how feedback has been addressed
        6. Explanation of how existing filter values have been incorporated
        7. Final filter JSON in the exact specified format

        Number of Solutions to Integrate: {len(solutions)}

        Solutions: {solutions}

        For each solution component, think through:
        - How does this map to our filter structure?
        - What are the explicit vs implicit requirements?
        - Are there any temporal aspects to consider (current vs past)?
        - What are the logical dependencies between different filters?
        - Are there any potential conflicts to resolve?
        - How does the feedback influence this component?
        - How do existing filter values influence this component?

        Show your complete reasoning process before providing the final JSON structure.
        """

        messages = [
            {"role": "system", "content": self.system_prompt},
            {"role": "user", "content": prompt}
        ]
        
        response = self.call_llm(messages)
        # Split the response into reasoning and JSON parts
        parts = response.split("FINAL FILTER JSON:")
        reasoning = parts[0].strip()
        
        # Parse the JSON string
        try:
            json_str = (response[response.find('{'):response.rfind('}') + 1])
            filter_structure = json.loads(json_str)
            
            return {
                "reasoning_and_analysis": reasoning,
                "filter_structure": filter_structure
            }
        except Exception as e:
            logger.error(f"Error parsing integration response: {e}")
            # Try to find any JSON in the response
            json_matches = re.findall(r'({.*})', response, re.DOTALL)
            for potential_json in json_matches:
                try:
                    filter_structure = json.loads(potential_json)
                    return {
                        "reasoning_and_analysis": reasoning,
                        "filter_structure": filter_structure,
                        "parsing_note": "Had to extract JSON using regex"
                    }
                except:
                    continue
            
            # If we still can't parse any JSON, return an error
            return {
                "error": str(e),
                "reasoning_and_analysis": reasoning,
                "raw_response": response
            }

# Updated Recruitment Query Processor with filter values and learnings
class FilterAwareRecruitmentQueryProcessor:
    def __init__(self, api_key: str):
        self.intent_agent = FilterAwareIntentAnalysisAgent(api_key)
        self.subproblem_agent = FilterAwareSubProblemAnalysisAgent(api_key)
        self.reflection_agent = SolutionReflectionAgent(api_key)
        self.integration_agent = IntegrationAgent(api_key)

    async def process_sub_problems(self, sub_problems: List[Dict], query_context: QueryContext) -> List[Dict]:
        # Create tasks for all sub-problems
        tasks = []
        for sub_problem_dict in sub_problems:
            sub_problem = SubProblem(**sub_problem_dict)
            task = self.subproblem_agent.solve_async(sub_problem, query_context)
            tasks.append(task)
        
        # Execute all tasks concurrently
        solutions = await asyncio.gather(*tasks)
        return solutions

    async def refine_solutions(self, solutions_needing_refinement: List[Dict], sub_problems: List[Dict], 
                               existing_solutions: List[Dict], query_context: QueryContext) -> List[Dict]:
        # Map sub-problem categories to original sub-problem dicts
        sub_problem_map = {sp["category"]: sp for sp in sub_problems}
        
        # Create a safe way to extract category from solutions
        def get_solution_category(solution):
            if "sub_problem_analysis" in solution:
                # First try the explicit category field we added
                if "category" in solution["sub_problem_analysis"]:
                    return solution["sub_problem_analysis"]["category"]
                
                # Then try to extract from original_context if available
                if "original_context" in solution["sub_problem_analysis"]:
                    context = solution["sub_problem_analysis"]["original_context"]
                    category_match = re.search(r"Category:\s*([^\n]+)", context)
                    if category_match:
                        return category_match.group(1).strip()
            
            # If we can't find it, return None
            return None
        
        # Create a mapping of categories to solutions
        solution_map = {}
        for solution in existing_solutions:
            category = get_solution_category(solution)
            if category:
                solution_map[category] = solution
        
        # Create tasks for refining solutions
        refinement_tasks = []
        for solution_info in solutions_needing_refinement:
            category = solution_info.get("sub_problem_category")
            if category and category in sub_problem_map:
                sub_problem = SubProblem(**sub_problem_map[category])
                
                # Add the feedback from reflection agent to the sub-problem
                if sub_problem.relevant_feedback:
                    sub_problem.relevant_feedback += f"\n\nADDITIONAL REFINEMENT FEEDBACK: {solution_info.get('feedback_for_agent', '')}"
                else:
                    sub_problem.relevant_feedback = f"REFINEMENT FEEDBACK: {solution_info.get('feedback_for_agent', '')}"
                
                task = self.subproblem_agent.solve_async(sub_problem, query_context)
                refinement_tasks.append((category, task))
            else:
                logger.warning(f"Could not find matching sub-problem for category: {category}")
        
        # Execute refinement tasks
        refined_solutions = []
        for category, task in refinement_tasks:
            try:
                refined_solution = await task
                # Add metadata about refinement
                if "sub_problem_analysis" in refined_solution:
                    refined_solution["sub_problem_analysis"]["refinement_metadata"] = {
                        "was_refined": True,
                        "refinement_reason": "Solution identified for improvement by reflection agent",
                        "original_solution_reference": f"Solution for category {category}"
                    }
                    # Ensure category is explicitly stored
                    refined_solution["sub_problem_analysis"]["category"] = category
                
                refined_solutions.append((category, refined_solution))
            except Exception as e:
                logger.error(f"Error refining solution for category {category}: {e}")
                traceback.print_exc()
        
        # Create the final list of solutions, replacing the ones that were refined
        final_solutions = list(existing_solutions)  # Make a copy
        
        for category, refined_solution in refined_solutions:
            # Find if there's an existing solution with this category
            replaced = False
            for i, solution in enumerate(final_solutions):
                if get_solution_category(solution) == category:
                    final_solutions[i] = refined_solution
                    replaced = True
                    break
            
            # If we didn't find a matching solution to replace, append the refined one
            if not replaced:
                final_solutions.append(refined_solution)
        
        return final_solutions

    async def process_query_async(self, query: str, feedback: Optional[str] = None, 
                                  filter_values: Optional[Dict[str, Any]] = None,
                                  learnings: Optional[str] = None) -> Dict:
        query_context = QueryContext(
            original_query=query,
            timestamp=datetime.now().isoformat(),
            query_id=f"query_{hash(query)}",
            feedback=feedback,
            filter_values=filter_values,
            learnings=learnings
        )
        
        logger.info(f"Processing query: {query_context.query_id}")
        logger.info(f"Feedback provided: {'Yes' if feedback else 'No'}")
        logger.info(f"Filter values provided: {'Yes' if filter_values else 'No'}")
        logger.info(f"Learnings provided: {'Yes' if learnings else 'No'}")

        try:
            # Step 1: Analyze intent and decompose, incorporating feedback, filter values, and learnings
            print('-'*47)
            print("Analyzing Intent and Decomposing the Problem (with feedback, filter values, and learnings)...")
            analysis = self.intent_agent.analyze(query_context)
            logger.info(f"Query decomposed into {len(analysis['sub_problems'])} sub-problems")

            # Step 2: Process all sub-problems concurrently, with relevant feedback, filter values, and learnings
            print('-'*47)
            print("Processing the Sub-Problems (with feedback, filter values, and learnings)...")
            solutions = await self.process_sub_problems(analysis['sub_problems'], query_context)
            logger.info(f"Generated initial solutions for all sub-problems")
            
            # Step 3: Use reflection agent to evaluate solutions and identify any needing refinement
            print('-'*47)
            print("Evaluating Solutions with Reflection Agent...")
            evaluation = await self.reflection_agent.evaluate_solutions(solutions, query_context)
            
            # Step 4: If there are solutions requiring refinement, process them
            if "solutions_requiring_iteration" in evaluation and evaluation["solutions_requiring_iteration"]:
                print('-'*47)
                print(f"Refining {len(evaluation['solutions_requiring_iteration'])} solutions that need improvement...")
                try:
                    solutions = await self.refine_solutions(
                        evaluation["solutions_requiring_iteration"], 
                        analysis['sub_problems'],
                        solutions,
                        query_context
                    )
                    logger.info(f"Refined solutions based on reflection feedback")
                except Exception as e:
                    logger.error(f"Error during solution refinement: {e}")
                    traceback.print_exc()
                    print("Continuing with original solutions despite refinement error")
            else:
                logger.info("No solutions required refinement")

            # Step 5: Integrate solutions
            print('-'*47)
            print("Integrating Solutions of Sub-Problems...")
            final_result = self.integration_agent.integrate(solutions, query_context)
            logger.info("Solutions integrated successfully")

            print('-'*47)
            print("Printing Final Result...")
            print(json.dumps(final_result, indent=2))

            return {
                "query_context": vars(query_context),
                "analysis": analysis,
                "solutions": solutions,
                "solution_evaluation": evaluation if "solutions_requiring_iteration" in evaluation else {"message": "No evaluation performed"},
                "final_result": final_result
            }

        except Exception as e:
            logger.error(f"Error processing query: {e}")
            traceback.print_exc()
            raise

# Modified main function to demonstrate feedback, filter values, and learnings
async def main():
    _ = load_dotenv()
    api_key = os.getenv("OPENAI_API_KEY")
    processor = FilterAwareRecruitmentQueryProcessor(api_key)
    
    # Example of filter values from a previous run
    example_filter_values = {
        "job_title/business_function": {
            "current": [{"title_name": "Sales Executive", "min_staff": 0, "max_staff": 50000000}],
            "past": [],
            "both": [],
            "event": "CURRENT"
        },
        "management_level": {
            "current": [],
            "past": [],
            "both": [],
            "event": ""
        },
        "location": {
            "current": [],
            "past": [],
            "both": [],
            "event": ""
        }
    }
    
    queries = [
        # {
        #     "query": "Find sales executives.",
        #     "feedback": "We need to focus on senior sales executives only.",
        #     "filter_values": example_filter_values,
        #     "learnings": ""
        # },
        # {
        #     "query": "Find VP-level leaders from companies in Western Europe with over 30 years of experience and expertise in corporate finance.",
        #     "feedback": "The results included too many people from Eastern Europe. Also, we need to see people who specifically have M&A experience.",
        #     "filter_values": None,
        #     "learnings": ""
        # },
        {
            "query": "Find me a Chief Marketing Officer with a strong track record in scaling AI-driven SaaS platforms. Preferably based in the U.S.",
            "feedback": "",  # "People who are not CMOs are appearing a lot. The results included too many people from US. Also, we need to see people who specifically have worked in sales in the past.",
            "learnings": """- The company filter can only generate well-known companies
                            - When finding profiles that identify people with strong track record in any field, the min company tenure should be 3
                            - The search engine treats individual keywords independently and may not understand relationships between terms (e.g., "scaling AI platforms" vs just "AI" and "platforms")
                            - Action verbs (like "scaling") may need to be emphasized more explicitly in profile_keywords to ensure relevant experience
                            - Job titles are interpreted literally, so including broader titles (like VP when looking for C-level) will bring in unwanted profiles
                            - The search engine doesn't automatically prioritize the most important aspects of a query - all keywords appear to have similar weight
                            - When looking for specific experience (like "scaling"), it's better to be explicit in profile_keywords rather than relying on company_keywords alone
                            - Broader search terms lead to more profiles but with lower precision - narrower terms yield fewer but more relevant results
                            """,
            "filter_values": None
        },
    ]
    
    query_results = []
    for query_item in queries:
        print('-'*47)
        print("QUERY: ", query_item["query"])
        print("FEEDBACK: ", query_item["feedback"])
        print("FILTER VALUES: ", "Provided" if query_item["filter_values"] else "None")
        print("LEARNINGS: ", "Provided" if query_item.get("learnings") else "None")
        
        try:
            result = await processor.process_query_async(
                query_item["query"], 
                query_item["feedback"],
                query_item["filter_values"],
                query_item.get("learnings", "")
            )
            query_results.append({
                "query": query_item["query"], 
                "feedback": query_item["feedback"],
                "filter_values": query_item["filter_values"],
                "learnings": query_item.get("learnings", ""),
                "result": result
            })
        except Exception as e:
            traceback.print_exc()
            print(f"Error processing query: {e}")
    
    return query_results

# If you just want to test a single query with feedback and filter values
async def test_single_query():
    _ = load_dotenv()
    api_key = os.getenv("OPENAI_API_KEY")
    processor = FilterAwareRecruitmentQueryProcessor(api_key)
    
    query = "Find VP-level leaders from AI companies in Western Europe with over 30 years of experience and expertise in corporate finance who have worked previously as Salesman."
    feedback = "The results included too many professionals from big tech companies like Google and Microsoft. We need to focus on smaller AI startups and scale-ups. Also, 'salesman' is too junior - we need people who were previously in sales management roles, not individual contributors."
    
    # Example filter values
    filter_values = {
        "job_title/business_function": {
            "current": [{"title_name": "VP", "min_staff": 0, "max_staff": 50000000}],
            "past": [{"title_name": "Salesman", "min_staff": 0, "max_staff": 50000000}],
            "both": [],
            "event": "CURRENT AND PAST"
        },
        "location": {
            "current": [{"continent": "Europe", "region": "Western Europe"}],
            "past": [],
            "both": [],
            "event": "CURRENT"
        },
        "total_working_years": {"min": 30, "max": null}
    }
    
    result = await processor.process_query_async(query, feedback, filter_values)
    return result

if __name__ == "__main__":
    results = await (main())
    # Or for testing single query:
    # result = asyncio.run(test_single_query())
    
    with open("filter_feedback_results.json", "w") as f:
        json.dump(results, f, indent=2)

INFO:__main__:Processing query: query_1263552615401363675
INFO:__main__:Feedback provided: No
INFO:__main__:Filter values provided: No
INFO:__main__:Learnings provided: Yes


-----------------------------------------------
QUERY:  Find me a Chief Marketing Officer with a strong track record in scaling AI-driven SaaS platforms. Preferably based in the U.S.
FEEDBACK:  
FILTER VALUES:  None
LEARNINGS:  Provided
-----------------------------------------------
Analyzing Intent and Decomposing the Problem (with feedback, filter values, and learnings)...
[
  {
    "role": "system",
    "content": "You are an expert recruitment query analyzer specializing in breaking down complex queries into ATOMIC logical sub-problems while prioritizing feedback incorporation and considering existing filter values.\n\n        Your PRIMARY RESPONSIBILITIES (in order of importance):\n        1. FEEDBACK FOCUS: Deeply analyze any provided feedback and make it the central guiding principle of your analysis\n        2. ATOMICITY: Break down queries into truly atomic sub-problems (each addressing ONE specific aspect only)\n        3. FILTER AWARENESS: Carefully consider existing filter

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:__main__:Query decomposed into 4 sub-problems


```json
{
    "main_intent": "To find a Chief Marketing Officer with a proven ability to scale AI-driven SaaS platforms, preferably located in the U.S.",
    "feedback_analysis": "No specific feedback was provided in this case, so the focus remains on the original query's details.",
    "filter_analysis": "No specific filters are provided in the query. However, the preference for U.S. location may imply a location filter if applicable.",
    "learnings_analysis": "The system learnings suggest prioritizing explicit experience descriptions, emphasizing action verbs like 'scaling', and ensuring a minimum tenure to affirm a strong track record.",
    "thinking": "The query was broken down into atomic sub-problems by isolating each distinct requirement: job title, location, experience in AI-driven SaaS, and the action of scaling.",
    "analysis_notes": [
        "The query primarily focuses on finding candidates with specific job titles and experience.",
        "The requirement for U.S. l

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"

...

-----------------------------------------------
Refining 4 solutions that need improvement...


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"

-----------------------------------------------
Integrating Solutions of Sub-Problems...
###############################################
[{'sub_problem_analysis': {'original_context': "Sub-problem Category: job_title\nSub-problem Description: Identify candidates with the job title 'Chief Marketing Officer'.\nSpecific Query Segment to Address: Chief Marketing Officer", 'thought_process': "The task is to filter candidates specifically holding the job title of 'Chief Marketing Officer'. The feedback emphasizes focusing on this exact title without including broader roles. System learnings highlight the need to avoid broader title inclusions to prevent irrelevant profiles.", 'interpreted_requirement': "Identify candidates with the exact job title of 'Chief Marketing Officer', while also considering common variations or abbreviations such as 'CMO'.", 'relevant_filters': {'identified_filters': ['JOB_TITLE'], 'rationale': 'The job title filter is relevant because it allows for precise matching

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:__main__:Solutions integrated successfully


### 1. DETAILED ANALYSIS

#### Job Title/Business Function
- **Analysis**: The requirement specifically seeks candidates with the job title "Chief Marketing Officer" (CMO), emphasizing precise matching without broader inclusions like VP or Director roles.
- **Temporal Aspect**: The focus is on the current role, as the query specifies finding a CMO, suggesting they should currently hold this position.
- **Feedback Influence**: Feedback emphasizes the exact title, aligning with the requirement to avoid broader roles.

#### Location
- **Analysis**: The requirement specifies candidates based in the U.S., suggesting a broad initial filter with the flexibility to refine based on specific regional feedback.
- **Temporal Aspect**: The location is current, as the query seeks candidates presently based in the U.S.
- **Feedback Influence**: No specific feedback was provided, but the broad U.S. focus aligns with the original requirement.

#### Experience
- **Analysis**: The focus is on candidates 